<a href="https://colab.research.google.com/github/GianFadiga/MCDA_FS/blob/main/mcda_py_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/GianFadiga/MCDA_FS/blob/main/MCDA_UT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# THE MCDA PROJECT: Criação da Ferramenta

## Início

#### Vamos ao código!

In [ ]:
'''
  Aqui toda a execução do código se inicia, não se esqueça de executar
  esta célula que é fundamental para o funcionamento do código
'''

# Importar as bibliotecas a serem utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
'''
  DICA: Divida os dados qualitativos em categorias (exemplo na CÉLULA 2),
  aqui está uma tabela para auxiliar no processo.
'''

# - Dividir em 5 categorias
# (0 - 20% (Nada desejável),
# 20 - 40% (Pouco desejável),
# 40 - 60% (Médio desejável),
# 60 - 80% (Bem desejável),
# 80 - 100% (Muito desejável))

'\n  DICA: Divida os dados qualitativos em categorias (exemplo na CÉLULA 2),\n  aqui está uma tabela para auxiliar no processo.\n'

# Realizando a montagem dos gráficos

## **CÉLULA 1 - DADOS NUMÉRICOS**

In [ ]:
import pandas as pd
import plotly.express as px

# Leitura do arquivo CSV com a codificação ISO-8859-1
teste_df = pd.read_csv('base_expandida.csv', encoding='ISO-8859-1')

# Extraindo o montante relacional e os tipos de dados
montante_relacional = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
teste_df = teste_df.drop([0, 1]).reset_index(drop=True)

# Separando os valores 'BOM' e 'NEUTRO'
valores_bom = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')

# Convertendo apenas as colunas numéricas para float
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom[col] = float(valores_bom[col])
        valores_neutro[col] = float(valores_neutro[col])
        teste_df[col] = teste_df[col].astype(float)

# Função para calcular a pontuação baseada na proximidade ao neutro e ao bom
def calcular_pontuacao(valor, bom, neutro, peso):
    if bom - neutro == 0:
        return 0
    elif valor >= neutro:
        return ((valor - neutro) / (bom - neutro)) * peso
    else:
        return -((neutro - valor) / neutro) * peso

# Aplicando a função de pontuação para cada atributo numérico
for col in montante_relacional.index:
    if tipos_dados[col] == 'number':
        teste_df[col + '_pontuacao'] = teste_df.apply(lambda x: calcular_pontuacao(x[col], valores_bom[col], valores_neutro[col], montante_relacional[col]), axis=1)

# Somando as pontuações para obter a pontuação total
teste_df['Pontuacao_Total'] = teste_df[[col + '_pontuacao' for col in montante_relacional.index if tipos_dados[col] == 'number']].sum(axis=1)

# Adicionando classificadores 'BOM' e 'NEUTRO' ao DataFrame
teste_df.loc[teste_df['Modelo'] == 'BOM', 'Color'] = 'Classificadores'
teste_df.loc[teste_df['Modelo'] == 'NEUTRO', 'Color'] = 'Classificadores'
teste_df.loc[~teste_df['Modelo'].isin(['BOM', 'NEUTRO']), 'Color'] = 'Desempenho'

# Ordenando os produtos pela pontuação total
teste_df = teste_df.sort_values(by='Pontuacao_Total', ascending=False).reset_index(drop=True)

# Identificando a pontuação máxima entre os produtos recomendados
pontuacao_maxima = teste_df[teste_df['Color'] == 'Desempenho']['Pontuacao_Total'].max()
produtos_recomendados = teste_df[(teste_df['Color'] == 'Desempenho') & (teste_df['Pontuacao_Total'] == pontuacao_maxima)]

# Exibindo todos os produtos recomendados com detalhes para leigos
for _, produto in produtos_recomendados.iterrows():
    print(f"Produto recomendado: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
    print("Pontos de vantagem e justificativas:")

    pontuacoes_detalhadas = []
    for col in montante_relacional.index:
        if tipos_dados[col] == 'number':
            pontuacao_atributo = produto[col + '_pontuacao']
            if pontuacao_atributo > 0:
                # Contexto adicional para leigos
                justificativa = (
                    f"O produto teve vantagem no atributo '{col}' onde o desempenho neste atributo está {pontuacao_atributo * 100:.0f}% mais próximo do ideal ('BOM') "
                    f"em comparação com o mínimo aceitável ('NEUTRO')."
                )
                pontuacoes_detalhadas.append(justificativa)

    # Exibindo as justificativas detalhadas para o produto
    for detalhe in pontuacoes_detalhadas:
        print(detalhe)
    print("\n")

# Preparando os dados para o gráfico
df_analise = teste_df.copy()

# Definindo cores baseadas nas pontuações
# df_analise['Color'] = df_analise['Pontuacao_Total'].apply(
#     lambda x: 'Classificadores' if x >= 0 else 'Desempenho'
# )

# Definindo as cores com base na pontuação total usando lambda
df_analise['Color'] = df_analise.apply(
    lambda row: 'Classificadores' if row['Modelo'] in ['BOM', 'NEUTRO'] else ('Pontuação Positiva' if row['Pontuacao_Total'] >= 0 else 'Pontuação Negativa'),
    axis=1
)

# Definindo o mapeamento de cores
color_mapping = {
    'Classificadores': 'Classificador',       # Para BOM e NEUTRO
    'Pontuação Positiva': 'Pontuação positiva',  # Para Desempenho
    'Pontuação Negativa': 'Pontuação negativa'   # Para pontuação negativa
}

# Mapeando as cores corretamente
df_analise['Color'] = df_analise['Color'].map(color_mapping)

# Plotando o gráfico
graph = px.bar(df_analise, x='Pontuacao_Total', y='Modelo', orientation='h',
               color='Color', text_auto=False, hover_name='Color',
               hover_data={'Color': False, 'Modelo': False, 'Pontuacao_Total': True},
               color_discrete_sequence=["cornflowerblue", "lightgreen", "palevioletred"])

graph.update_layout(yaxis={'categoryorder': 'total ascending'})
graph.update_xaxes(showticklabels=True)
graph.update_yaxes(showticklabels=True)
graph.update_coloraxes(showscale=False)
graph.update_layout(xaxis_title='')

# Adicionando a linha neutra
graph.add_vline(x=0, line_dash='dash', line_color='black')
graph.add_annotation(
    x=0,
    y=len(df_analise),
    xref="x",
    yref="y",
    text="Linha Neutra<br>(Mínimo Aceitável)",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#000000"
    ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor='white',
    opacity=1
)

graph.update_layout(title='Comparação com pontuação final',
                    legend_title_text='Legenda da relação de cores')
graph.update_layout(hovermode='y unified')

graph.show()


Produto recomendado: Clevo com pontuação total de 5.45
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' onde o desempenho neste atributo está 450% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' onde o desempenho neste atributo está 95% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').


Produto recomendado: Eurocom com pontuação total de 5.45
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' onde o desempenho neste atributo está 450% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' onde o desempenho neste atributo está 95% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').




In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

# Leitura do arquivo CSV com a codificação UTF-8
teste_df = pd.read_csv('base_new_2.csv', encoding='UTF-8', sep=';')

# Extraindo o montante relacional e os tipos de dados
montante_relacional = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
proporcionalidade = teste_df.iloc[2].drop('Modelo')

# Separando os DataFrames de 'BOM', 'NEUTRO' e os demais para cálculo
valores_bom_df = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro_df = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')
calculo_df = teste_df.drop([0, 1, 2], errors='ignore').reset_index(drop=True)

# Convertendo os tipos de dados conforme especificado
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom_df[col] = float(valores_bom_df[col])
        valores_neutro_df[col] = float(valores_neutro_df[col])
        calculo_df[col] = calculo_df[col].astype(float)
    elif tipos_dados[col] == 'boolean':
        valores_bom_df[col] = valores_bom_df[col] == 'TRUE'
        valores_neutro_df[col] = valores_neutro_df[col] == 'TRUE'
        calculo_df[col] = calculo_df[col].map({'TRUE': True, 'FALSE': False})

# Função para calcular a pontuação baseada na proximidade ao neutro e ao bom
def calcular_pontuacao(valor, bom, neutro, peso, proporcionalidade_tipo):
    if bom == neutro:
        return 0
    elif proporcionalidade_tipo == 'proportional':
        if isinstance(valor, bool):
            valor_numeric = 1.0 if valor == bom else 0.0
            bom_numeric = 1.0 if bom else 0.0
            neutro_numeric = 1.0 if neutro else 0.0
            if bom_numeric == neutro_numeric:
                return 0
            elif valor_numeric >= neutro_numeric:
                return ((valor_numeric - neutro_numeric) / (bom_numeric - neutro_numeric)) * peso
            else:
                return -((neutro_numeric - valor_numeric) / neutro_numeric) * peso
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            bom_numeric = 1.0 if bom else 0.0
            neutro_numeric = 1.0 if neutro else 0.0
            if bom_numeric == neutro_numeric:
                return 0
            elif valor >= neutro_numeric:
                return ((valor - neutro_numeric) / (bom_numeric - neutro_numeric)) * peso
            else:
                return -((neutro_numeric - valor) / neutro_numeric) * peso
        elif valor >= neutro:
            return ((valor - neutro) / (bom - neutro)) * peso
        else:
            return -((neutro - valor) / neutro) * peso
    elif proporcionalidade_tipo == 'i_proportional':
        if isinstance(valor, bool):
            valor_numeric = 1.0 if valor == bom else 0.0
            bom_numeric = 1.0 if bom else 0.0
            neutro_numeric = 1.0 if neutro else 0.0
            if bom_numeric == neutro_numeric:
                return 0
            elif valor_numeric <= neutro_numeric:
                return ((neutro_numeric - valor_numeric) / neutro_numeric) * peso
            else:
                return -((valor_numeric - neutro_numeric) / (bom_numeric - neutro_numeric)) * peso
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            bom_numeric = 1.0 if bom else 0.0
            neutro_numeric = 1.0 if neutro else 0.0
            if bom_numeric == neutro_numeric:
                return 0
            elif valor <= neutro_numeric:
                return ((neutro_numeric - valor) / neutro_numeric) * peso
            else:
                return -((valor - neutro_numeric) / (bom_numeric - neutro_numeric)) * peso
        elif bom < neutro:  # Quanto menor, melhor: BOM é o limite inferior ideal, NEUTRO o superior aceitável
            if valor <= bom:
                return peso  # Pontuação máxima se igual ou melhor que o ideal
            elif bom < valor <= neutro:
                return ((neutro - valor) / (neutro - bom)) * peso  # Pontuação diminui de peso para 0
            elif valor > neutro:
                return -((valor - neutro) / neutro) * peso # Pontuação negativa se pior que o aceitável
        elif bom > neutro: # Lógica para o caso onde o 'BOM' é um valor indesejavelmente alto (raro para "menor é melhor")
            if valor >= bom:
                return peso
            elif neutro <= valor < bom:
                return ((valor - neutro) / (bom - neutro)) * peso
            elif valor < neutro:
                return -((neutro - valor) / neutro) * peso
        else: # bom == neutro, já tratado no início
            return 0
    else:
        return 0
        return 0

# Aplicando a função de pontuação para cada atributo relevante
for col in montante_relacional.index:
    if tipos_dados[col] in ['number', 'boolean']:
        calculo_df[col + '_pontuacao'] = calculo_df.apply(
            lambda x: calcular_pontuacao(x[col], valores_bom_df[col], valores_neutro_df[col], montante_relacional[col], proporcionalidade[col]),
            axis=1
        )

# Somando as pontuações para obter a pontuação total
colunas_pontuacao = [col + '_pontuacao' for col in montante_relacional.index if tipos_dados[col] in ['number', 'boolean']]
calculo_df['Pontuacao_Total'] = calculo_df[colunas_pontuacao].sum(axis=1)

# DataFrame para análise (incluindo BOM e NEUTRO com suas respectivas "pontuações")
df_analise = pd.concat([calculo_df[['Modelo', 'Pontuacao_Total']]], ignore_index=True)

# Adicionando uma coluna para colorir os pontos no gráfico
df_analise['Color'] = df_analise['Modelo'].apply(lambda x: 'Classificadores' if x in ['BOM', 'NEUTRO'] else ('Pontuação Positiva' if pd.notna(df_analise.loc[df_analise['Modelo'] == x, 'Pontuacao_Total'].iloc[0]) and df_analise.loc[df_analise['Modelo'] == x, 'Pontuacao_Total'].iloc[0] >= 0 else 'Pontuação Negativa'))

# Definindo o mapeamento de cores
color_mapping = {
    'Classificadores': 'Classificador',
    'Pontuação Positiva': 'Pontuação positiva',
    'Pontuação Negativa': 'Pontuação negativa'
}

# Mapeando as cores corretamente
df_analise['Color'] = df_analise['Color'].map(color_mapping)

# Ordenando os produtos pela pontuação total (mantendo BOM e NEUTRO no final para visualização)
df_analise_sorted = df_analise.sort_values(by='Pontuacao_Total', ascending=False, na_position='last').reset_index(drop=True)

# Identificando a pontuação máxima entre os produtos recomendados (excluindo BOM e NEUTRO)
produtos_desempenho = calculo_df[~calculo_df['Modelo'].isin(['BOM', 'NEUTRO'])]
if not produtos_desempenho.empty:
    pontuacao_maxima = produtos_desempenho['Pontuacao_Total'].max()
    produtos_recomendados = produtos_desempenho[produtos_desempenho['Pontuacao_Total'] == pontuacao_maxima]

    # Exibindo todos os produtos recomendados com detalhes para leigos
    print("Produtos recomendados:")
    for _, produto in produtos_recomendados.iterrows():
        print(f"Produto: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
        print("Pontos de vantagem e justificativas:")

        pontuacoes_detalhadas = []
        for col in montante_relacional.index:
            if tipos_dados[col] in ['number', 'boolean']:
                valor_produto = produto[col]
                bom_valor = valores_bom_df[col]
                neutro_valor = valores_neutro_df[col]
                pontuacao_atributo = calcular_pontuacao(
                    valor_produto,
                    bom_valor,
                    neutro_valor,
                    montante_relacional[col],
                    proporcionalidade[col]
                )
                if proporcionalidade[col] == 'proportional':
                    # ... (mesma lógica de justificativa para proporcional)
                    if isinstance(bom_valor, bool) and isinstance(neutro_valor, bool):
                        if valor_produto == bom_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto atende ao critério ideal ('BOM') para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto != neutro_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto não atende ao critério mínimo aceitável ('NEUTRO') para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                    elif pontuacao_atributo > 0:
                        justificativa = (
                            f"O produto teve vantagem no atributo '{col}' onde o desempenho neste atributo está {pontuacao_atributo * 100:.0f}% mais próximo do ideal ('BOM') "
                            f"em comparação com o mínimo aceitável ('NEUTRO')."
                        )
                        pontuacoes_detalhadas.append(justificativa)
                    elif pontuacao_atributo < 0:
                        justificativa = (
                            f"O produto ficou abaixo no atributo '{col}' onde o desempenho neste atributo está {-pontuacao_atributo * (100 * (1 if isinstance(neutro_valor, bool) else neutro_valor)):.0f}% abaixo do mínimo aceitável ('NEUTRO')."
                        )
                        pontuacoes_detalhadas.append(justificativa)
                elif proporcionalidade[col] == 'i_proportional':
                    # ... (mesma lógica de justificativa para inversamente proporcional)
                    if isinstance(bom_valor, bool) and isinstance(neutro_valor, bool):
                        if valor_produto == neutro_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto atende ao critério ideal ('NEUTRO' para atributos inversamente proporcionais) para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto != bom_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto não atinge o nível menos desejado ('BOM' para atributos inversamente proporcionais) para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                    elif bom_valor > neutro_valor: # Ideal é menor
                        if valor_produto <= neutro_valor:
                            justificativa = f"O produto atingiu o ideal ('NEUTRO') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto > bom_valor:
                            justificativa = f"O produto está no nível menos desejado ('BOM') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo > 0:
                            justificativa = f"O produto está performando bem no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo < 0:
                            justificativa = f"O produto precisa melhorar no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                    elif bom_valor < neutro_valor: # Ideal é maior
                        if valor_produto >= neutro_valor:
                            justificativa = f"O produto atingiu o ideal ('NEUTRO') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto < bom_valor:
                            justificativa = f"O produto está no nível menos desejado ('BOM') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo > 0:
                            justificativa = f"O produto está performando bem no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo < 0:
                            justificativa = f"O produto precisa melhorar no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)

        for detalhe in pontuacoes_detalhadas:
            print(detalhe)
        print("\n")
else:
    print("Nenhum produto de desempenho encontrado para recomendação.")

# Plotando o gráfico
graph = px.bar(df_analise_sorted, x='Pontuacao_Total', y='Modelo', orientation='h',
             color='Color', text_auto=False, hover_name='Modelo',
             hover_data={'Color': False, 'Modelo': False, 'Pontuacao_Total': True},
             color_discrete_sequence=["cornflowerblue", "lightgreen", "palevioletred"])

graph.update_layout(yaxis={'categoryorder': 'array', 'categoryarray': df_analise_sorted['Modelo'].tolist()[::-1]})
graph.update_xaxes(showticklabels=True)
graph.update_yaxes(showticklabels=True)
graph.update_coloraxes(showscale=False)
graph.update_layout(xaxis_title='')

# Adicionando a linha neutra
graph.add_vline(x=0, line_dash='dash', line_color='black')
graph.add_annotation(
    x=0,
    y=len(df_analise_sorted) - 1,
    xref="x",
    yref="y",
    text="Linha Neutra<br>(Mínimo Aceitável)",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#000000"
    ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor='white',
    opacity=1
)

graph.update_layout(title='Comparação com pontuação final',
                    legend_title_text='Legenda da relação de cores')
graph.update_layout(hovermode='y unified')

graph.show()

Produtos recomendados:
Produto: Apple com pontuação total de 1.51
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' onde o desempenho neste atributo está 90% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' onde o desempenho neste atributo está 21% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto atingiu o ideal ('NEUTRO') para o atributo 'Tamanho de tela' (inversamente proporcional).
O produto teve vantagem no atributo 'Bateria mAh' onde o desempenho neste atributo está 30% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto atende ao critério ideal ('BOM') para o atributo 'Bluetooth'.




### Teste Carro

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

# Leitura do arquivo CSV com a codificação UTF-8
teste_df = pd.read_csv('base_3.csv', encoding='UTF-8', sep=';')

# Extraindo o montante relacional e os tipos de dados
montante_relacional = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
proporcionalidade = teste_df.iloc[2].drop('Modelo')

# Separando os DataFrames de 'BOM', 'NEUTRO' e os demais para cálculo
valores_bom_df = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro_df = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')
calculo_df = teste_df.drop([0, 1, 2], errors='ignore').reset_index(drop=True)

# Convertendo os tipos de dados conforme especificado
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom_df[col] = float(valores_bom_df[col])
        valores_neutro_df[col] = float(valores_neutro_df[col])
        calculo_df[col] = calculo_df[col].astype(float)
    elif tipos_dados[col] == 'boolean':
        valores_bom_df[col] = valores_bom_df[col] == 'TRUE'
        valores_neutro_df[col] = valores_neutro_df[col] == 'TRUE'
        calculo_df[col] = calculo_df[col].map({'TRUE': True, 'FALSE': False})

# Calculo da pontuação (inclui boolean e proporcionalidades)
def calcular_pontuacao(valor, bom, neutro, peso, proporcionalidade_tipo):
    if bom == neutro:
        return 0
    elif proporcionalidade_tipo == 'proportional':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso  # Caso inesperado
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif bom == neutro:
            return 0
        elif valor >= neutro:
            return ((valor - neutro) / (bom - neutro)) * peso
        else:
            return -((neutro - valor) / neutro) * peso
    elif proporcionalidade_tipo == 'i_proportional':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif bom < neutro:
            if valor <= bom:
                return peso
            elif bom < valor <= neutro:
                return ((neutro - valor) / (neutro - bom)) * peso
            elif valor > neutro:
                return -((valor - neutro) / neutro) * peso
        elif bom > neutro:
            if valor >= bom:
                return peso
            elif neutro <= valor < bom:
                return ((valor - neutro) / (bom - neutro)) * peso
            elif valor < neutro:
                return -((neutro - valor) / neutro) * peso
        else:
            return 0
    elif proporcionalidade_tipo == 'boolean':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso # Caso inesperado
        else:
            return 0 # Tratar se o tipo de dado não for booleano
    else:
        return 0

# Aplicando a função de pontuação para cada atributo relevante
for col in montante_relacional.index:
    if tipos_dados[col] in ['number', 'boolean']:
        calculo_df[col + '_pontuacao'] = calculo_df.apply(
            lambda x: calcular_pontuacao(x[col], valores_bom_df[col], valores_neutro_df[col], montante_relacional[col], proporcionalidade[col]),
            axis=1
        )

# Somando as pontuações para obter a pontuação total
colunas_pontuacao = [col + '_pontuacao' for col in montante_relacional.index if tipos_dados[col] in ['number', 'boolean']]
calculo_df['Pontuacao_Total'] = calculo_df[colunas_pontuacao].sum(axis=1)

# DataFrame para análise (incluindo BOM e NEUTRO com suas respectivas "pontuações")
df_analise = pd.concat([calculo_df[['Modelo', 'Pontuacao_Total']]], ignore_index=True)

# Adicionando uma coluna para colorir os pontos no gráfico
df_analise['Color'] = df_analise['Modelo'].apply(lambda x: 'Classificadores' if x in ['BOM', 'NEUTRO'] else ('Pontuação Positiva' if pd.notna(df_analise.loc[df_analise['Modelo'] == x, 'Pontuacao_Total'].iloc[0]) and df_analise.loc[df_analise['Modelo'] == x, 'Pontuacao_Total'].iloc[0] >= 0 else 'Pontuação Negativa'))

# Definindo o mapeamento de cores
color_mapping = {
    'Classificadores': 'Classificador',
    'Pontuação Positiva': 'Pontuação positiva',
    'Pontuação Negativa': 'Pontuação negativa'
}

# Mapeando as cores corretamente
df_analise['Color'] = df_analise['Color'].map(color_mapping)

# Ordenando os produtos pela pontuação total (mantendo BOM e NEUTRO no final para visualização)
df_analise_sorted = df_analise.sort_values(by='Pontuacao_Total', ascending=False, na_position='last').reset_index(drop=True)

# Identificando a pontuação máxima entre os produtos recomendados (excluindo BOM e NEUTRO)
produtos_desempenho = calculo_df[~calculo_df['Modelo'].isin(['BOM', 'NEUTRO'])]
if not produtos_desempenho.empty:
    pontuacao_maxima = produtos_desempenho['Pontuacao_Total'].max()
    produtos_recomendados = produtos_desempenho[produtos_desempenho['Pontuacao_Total'] == pontuacao_maxima]

    # Exibindo todos os produtos recomendados com detalhes para leigos
    print("Produtos recomendados:")
    for _, produto in produtos_recomendados.iterrows():
        print(f"Produto: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
        print("Pontos de vantagem e justificativas:")

        pontuacoes_detalhadas = []
        for col in montante_relacional.index:
            if tipos_dados[col] in ['number', 'boolean']:
                valor_produto = produto[col]
                bom_valor = valores_bom_df[col]
                neutro_valor = valores_neutro_df[col]
                pontuacao_atributo = calcular_pontuacao(
                    valor_produto,
                    bom_valor,
                    neutro_valor,
                    montante_relacional[col],
                    proporcionalidade[col]
                )
                if proporcionalidade[col] == 'proportional':
                    # ... (mesma lógica de justificativa para proporcional)
                    if isinstance(bom_valor, bool) and isinstance(neutro_valor, bool):
                        if valor_produto == bom_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto atende ao critério ideal ('BOM') para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto != neutro_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto não atende ao critério mínimo aceitável ('NEUTRO') para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                    elif pontuacao_atributo > 0:
                        justificativa = (
                            f"O produto teve vantagem no atributo '{col}' onde o desempenho neste atributo está {pontuacao_atributo * 100:.0f}% mais próximo do ideal ('BOM') "
                            f"em comparação com o mínimo aceitável ('NEUTRO')."
                        )
                        pontuacoes_detalhadas.append(justificativa)
                    elif pontuacao_atributo < 0:
                        justificativa = (
                            f"O produto ficou abaixo no atributo '{col}' onde o desempenho neste atributo está {-pontuacao_atributo * (100 * (1 if isinstance(neutro_valor, bool) else neutro_valor)):.0f}% abaixo do mínimo aceitável ('NEUTRO')."
                        )
                        pontuacoes_detalhadas.append(justificativa)
                elif proporcionalidade[col] == 'i_proportional':
                    # ... (mesma lógica de justificativa para inversamente proporcional)
                    if isinstance(bom_valor, bool) and isinstance(neutro_valor, bool):
                        if valor_produto == neutro_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto atende ao critério ideal ('NEUTRO' para atributos inversamente proporcionais) para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto != bom_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto não atinge o nível menos desejado ('BOM' para atributos inversamente proporcionais) para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                    elif bom_valor > neutro_valor: # Ideal é menor
                        if valor_produto <= neutro_valor:
                            justificativa = f"O produto atingiu o ideal ('NEUTRO') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto > bom_valor:
                            justificativa = f"O produto está no nível menos desejado ('BOM') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo > 0:
                            justificativa = f"O produto está performando bem no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo < 0:
                            justificativa = f"O produto precisa melhorar no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                    elif bom_valor < neutro_valor: # Ideal é maior
                        if valor_produto >= neutro_valor:
                            justificativa = f"O produto atingiu o ideal ('NEUTRO') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto < bom_valor:
                            justificativa = f"O produto está no nível menos desejado ('BOM') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo > 0:
                            justificativa = f"O produto está performando bem no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo < 0:
                            justificativa = f"O produto precisa melhorar no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)

        for detalhe in pontuacoes_detalhadas:
            print(detalhe)
        print("\n")
else:
    print("Nenhum produto de desempenho encontrado para recomendação.")

# Plotando o gráfico
graph = px.bar(df_analise_sorted, x='Pontuacao_Total', y='Modelo', orientation='h',
             color='Color', text_auto=False, hover_name='Modelo',
             hover_data={'Color': False, 'Modelo': False, 'Pontuacao_Total': True},
             color_discrete_sequence=["cornflowerblue", "lightgreen", "palevioletred"])

graph.update_layout(yaxis={'categoryorder': 'array', 'categoryarray': df_analise_sorted['Modelo'].tolist()[::-1]})
graph.update_xaxes(showticklabels=True)
graph.update_yaxes(showticklabels=True)
graph.update_coloraxes(showscale=False)
graph.update_layout(xaxis_title='')

# Adicionando a linha neutra
graph.add_vline(x=0, line_dash='dash', line_color='black')
graph.add_annotation(
    x=0,
    y=len(df_analise_sorted) - 1,
    xref="x",
    yref="y",
    text="Linha Neutra<br>(Mínimo Aceitável)",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#000000"
    ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor='white',
    opacity=1
)

graph.update_layout(title='Comparação com pontuação final',
                    legend_title_text='Legenda da relação de cores')
graph.update_layout(hovermode='y unified')

graph.show()

Produtos recomendados:
Produto: Honda City com pontuação total de 0.53
Pontos de vantagem e justificativas:
O produto atingiu o ideal ('NEUTRO') para o atributo 'Preço (R$)' (inversamente proporcional).
O produto teve vantagem no atributo 'Consumo (km/l)' onde o desempenho neste atributo está 4% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Potência (cv)' onde o desempenho neste atributo está 12% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Porta-malas (litros)' onde o desempenho neste atributo está 10% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Segurança (estrelas)' onde o desempenho neste atributo está 20% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').




In [ ]:
teste_df.head()

,Modelo,Preço (R$),Consumo (km/l),Potência (cv),Porta-malas (litros),Segurança (estrelas),Ar Condicionado,Câmbio Automático,Cor
0,NaN,0.3,0.1,0.1,0.05,0.2,0.1,0.1,0.05
1,NaN,number,number,number,number,number,boolean,boolean,string
2,NaN,i_proportional,proportional,proportional,proportional,proportional,boolean,boolean,NaN
3,BOM,60000,15,120,400,5,TRUE,TRUE,Branco
4,NEUTRO,85000,10,90,300,3,TRUE,FALSE,Prata


In [ ]:
calculo_df.head(15)

,Modelo,Preço (R$),Consumo (km/l),Potência (cv),Porta-malas (litros),Segurança (estrelas),Ar Condicionado,Câmbio Automático,Cor,Preço (R$)_pontuacao,Consumo (km/l)_pontuacao,Potência (cv)_pontuacao,Porta-malas (litros)_pontuacao,Segurança (estrelas)_pontuacao,Ar Condicionado_pontuacao,Câmbio Automático_pontuacao,Pontuacao_Total
0,BOM,60000.0,15.0,120.0,400.0,5.0,True,True,Branco,0.300000,0.100,0.100000,0.050000,0.2,0,0.1,0.850000
1,NEUTRO,85000.0,10.0,90.0,300.0,3.0,True,False,Prata,0.000000,0.000,0.000000,0.000000,0.0,0,0.0,0.000000
2,Fiat Argo,75000.0,13.5,109.0,350.0,4.0,True,False,Vermelho,0.120000,0.070,0.063333,0.025000,0.1,0,0.0,0.378333
3,Volkswagen Polo,82000.0,12.8,116.0,320.0,5.0,True,False,Azul,0.036000,0.056,0.086667,0.010000,0.2,0,0.0,0.388667
4,Chevrolet Onix,78000.0,14.2,106.0,305.0,4.0,True,False,Preto,0.084000,0.084,0.053333,0.002500,0.1,0,0.0,0.323833
5,Hyundai HB20,72000.0,13.8,101.0,300.0,4.0,True,False,Branco,0.156000,0.076,0.036667,0.000000,0.1,0,0.0,0.368667
6,Renault Kwid,65000.0,15.5,70.0,290.0,3.0,True,False,Prata,0.240000,0.110,-0.022222,-0.001667,0.0,0,0.0,0.326111
7,Toyota Yaris,88000.0,12.5,110.0,310.0,4.0,True,True,Cinza,-0.010588,0.050,0.066667,0.005000,0.1,0,0.1,0.311078
8,Honda City,92000.0,11.8,126.0,500.0,5.0,True,True,Azul,-0.024706,0.036,0.120000,0.100000,0.2,0,0.1,0.531294
9,Peugeot 208,80000.0,13.0,113.0,315.0,4.0,True,False,Vermelho,0.060000,0.060,0.076667,0.007500,0.1,0,0.0,0.304167


## **CÉLULA 2 - DADOS NÃO NUMÉRICOS**

In [11]:
df = pd.DataFrame()

# Situação/objeto exemplo: compra de um celular
# Começar considerando cores como categoria a ser analisada
# Considerar Vermelho como Muito desejável, Preto como Neutro e Verde como Nada desejável

# A ideia é a criação de diversos métodos de análise e com gráficos a serem analisados
# No caso, cada cor receberia um slider para que o usuário selecione a sua importância
# Considerando valores entre os intervalos dados (trabalhando na base de 0 até 100, logo, números decimais)
# Onde 0 é o mínimo desejável e 1 é o máximo desejável

'''
  Instruções de uso:
    - Com o dataframe já carregado da CÉLULA 1, basta alterar os dados de teste
    (placeholder) aqui presentes e executar o código.
    DICA: Crie cópias da célula, assim execute o código uma vez para cada dado
    qualitativo que desejar comparar.
'''

# Altere aqui! - Início
df['Cores'] = ['Vermelho', 'Verde', 'Branco', 'Preto', 'BOM', 'NEUTRO']
df['Valor Cores'] = [90, 10, 40, 50, 80, 50]
# Altere aqui! 0 Final

df['Tam_scatter'] = [1]*len(df)
symbols = ['diamond-wide']*len(df)

df['Valor X'] = [0]*len(df)

val_neutro = df['Valor Cores'][df.index[df['Cores'] == 'NEUTRO'][0]]
val_bom = df['Valor Cores'][df.index[df['Cores'] == 'BOM'][0]]

df['Definicao'] = pd.Series(dtype='str')
for i in range(len(df)):
    if df['Cores'][i] == 'NEUTRO' or df['Cores'][i] == 'BOM':
        df.loc[i, 'Definicao'] = 'Classificador'
    else:
        df['Definicao'] = 'Desempenho'
# Diversas cores para daltônicos

cores1 = ['red', 'blue']
cores2 = ['pink', 'green']
cores3 = ['orange', 'purple']
cores4 = ['orange', 'gray']

graph = px.scatter(df, x='Valor X', y='Valor Cores', title='Comparação com Indicadores de Desempenho e Classificadores',
                   color='Cores', symbol_sequence=symbols,
                   hover_data={'Cores':False, 'Valor Cores':False, 'Valor X':False,
                               'Valor Referencial': (df['Cores'])})

graph.update_traces(marker={'size': 20})
graph.update_xaxes(fixedrange=True, title=None)
graph.update_yaxes(fixedrange=True, title=None)

for i in range(len(df)):
    if df['Definicao'][i] == 'Classificador' and df['Cores'][i] == 'NEUTRO':
        graph.add_hline(y=df['Valor Cores'][i], line_color=cores3[1], line_width=3, x0=0.3, x1=0.7)
    elif df['Definicao'][i] == 'Classificador' and df['Cores'][i] == 'BOM':
        graph.add_hline(y=df['Valor Cores'][i], line_color=cores3[0], line_width=3, x0=0.3, x1=0.7)


graph.update_xaxes(showticklabels=False, showgrid=False)
graph.update_yaxes(showticklabels=False, showgrid=False)

graph.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=1,
    title_text='Variáveis e Cores'
))

graph.add_vline(x=0, y0=0, y1=1)
graph.add_annotation(
    x=-0.2,
    y=110,
    xref='x',
    yref='y',
    text=f'Linha Neutra<br>(Mínimo Aceitável)',
    showarrow=False,
    font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor=cores3[1],
        opacity=1
)
graph.add_annotation(
    x=0.2,
    y=110,
    xref='x',
    yref='y',
    text=f'Linha Bom<br>(Muito Desejável)',
    showarrow=False,
    font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor=cores3[0],
        opacity=1
)
graph.show()

## Teste 2


In [14]:
import pandas as pd
import plotly.express as px
import numpy as np

# Leitura do arquivo CSV
teste_df = pd.read_csv('base_carro.csv', encoding='UTF-8', sep=',')

# Extraindo os pesos e tipos de dados
pesos = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
proporcionalidade = teste_df.iloc[2].drop('Modelo')

# Separando os DataFrames de 'BOM', 'NEUTRO' e os demais para cálculo
valores_bom_df = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro_df = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')
calculo_df = teste_df.drop([0, 1, 2], errors='ignore').reset_index(drop=True)

# Convertendo os tipos de dados
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom_df[col] = float(valores_bom_df[col])
        valores_neutro_df[col] = float(valores_neutro_df[col])
        calculo_df[col] = calculo_df[col].astype(float)
    elif tipos_dados[col] == 'boolean':
        valores_bom_df[col] = valores_bom_df[col] == 'TRUE'
        valores_neutro_df[col] = valores_neutro_df[col] == 'TRUE'
        calculo_df[col] = calculo_df[col].map({'TRUE': True, 'FALSE': False})

# Função de pontuação para dados numéricos e booleanos
def calcular_pontuacao(valor, bom, neutro, peso, proporcionalidade_tipo):
    if bom == neutro:
        return 0
    elif proporcionalidade_tipo == 'proportional':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif bom == neutro:
            return 0
        elif valor >= neutro:
            return ((valor - neutro) / (bom - neutro)) * peso
        else:
            return -((neutro - valor) / neutro) * peso
    elif proporcionalidade_tipo == 'i_proportional':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif bom < neutro:
            if valor <= bom:
                return peso
            elif bom < valor <= neutro:
                return ((neutro - valor) / (neutro - bom)) * peso
            elif valor > neutro:
                return -((valor - neutro) / neutro) * peso
        elif bom > neutro:
            if valor >= bom:
                return peso
            elif neutro <= valor < bom:
                return ((valor - neutro) / (bom - neutro)) * peso
            elif valor < neutro:
                return -((neutro - valor) / neutro) * peso
        else:
            return 0
    elif proporcionalidade_tipo == 'boolean':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        else:
            return 0
    else:
        return 0

# Função de pontuação para dados de string (cor)
def calcular_pontuacao_cor(cor, peso):
    # Valores de exemplo - podem ser ajustados conforme necessidade
    valores_cores = {
        'Branco': 0.05,
        'Vermelho': 0.01,
        'Azul': 0.02,
        'Preto': 0.01,
        'Prata': 0.04,
        'Cinza': 0.03
    }

    valor_bom = valores_bom_df['Cor']
    valor_neutro = valores_neutro_df['Cor']

    if cor == valor_bom:
        return peso * valores_cores.get(cor, 0.01)
    elif cor == valor_neutro:
        return 0
    else:
        return -peso * 0.01  # Penalização padrão para cores não especificadas

# Aplicando a função de pontuação para atributos numéricos e booleanos
for col in pesos.index:
    if tipos_dados[col] in ['number', 'boolean']:
        calculo_df[col + '_pontuacao'] = calculo_df.apply(
            lambda x: calcular_pontuacao(x[col], valores_bom_df[col], valores_neutro_df[col], pesos[col], proporcionalidade[col]),
            axis=1
        )

# Aplicando a função de pontuação para a cor (string)
if 'Cor' in pesos.index and tipos_dados['Cor'] == 'string':
    calculo_df['Cor_pontuacao'] = calculo_df.apply(
        lambda x: calcular_pontuacao_cor(x['Cor'], pesos['Cor']),
        axis=1
    )

# Somando as pontuações
colunas_pontuacao = [col + '_pontuacao' for col in pesos.index if tipos_dados[col] in ['number', 'boolean', 'string']]
calculo_df['Pontuacao_Total'] = calculo_df[colunas_pontuacao].sum(axis=1)

# DataFrame para análise
df_analise = calculo_df[['Modelo', 'Pontuacao_Total']].copy()

# Adicionando uma coluna para colorir os pontos no gráfico
df_analise['Color'] = df_analise['Modelo'].apply(
    lambda x: 'Classificadores' if x in ['BOM', 'NEUTRO'] else
    ('Pontuação Positiva' if x in df_analise['Modelo'] and df_analise.loc[df_analise['Modelo'] == x, 'Pontuacao_Total'].iloc[0] >= 0
     else 'Pontuação Negativa')
)

# Mapeamento de cores
color_mapping = {
    'Classificadores': 'Classificador',
    'Pontuação Positiva': 'Pontuação positiva',
    'Pontuação Negativa': 'Pontuação negativa'
}

df_analise['Color'] = df_analise['Color'].map(color_mapping)
df_analise_sorted = df_analise.sort_values(by='Pontuacao_Total', ascending=False, na_position='last').reset_index(drop=True)

# Gráfico 1: Barra com a pontuação total (similar ao original)
graph1 = px.bar(df_analise_sorted, x='Pontuacao_Total', y='Modelo', orientation='h',
               color='Color', text_auto=False, hover_name='Modelo',
               hover_data={'Color': False, 'Modelo': False, 'Pontuacao_Total': True},
               color_discrete_sequence=["cornflowerblue", "lightgreen", "palevioletred"],
               title='Pontuação Total dos Modelos')

graph1.update_layout(yaxis={'categoryorder': 'array', 'categoryarray': df_analise_sorted['Modelo'].tolist()[::-1]})
graph1.update_xaxes(showticklabels=True)
graph1.update_yaxes(showticklabels=True)
graph1.update_coloraxes(showscale=False)
graph1.update_layout(xaxis_title='')

graph1.add_vline(x=0, line_dash='dash', line_color='black')
graph1.add_annotation(
    x=0,
    y=len(df_analise_sorted) - 1,
    xref="x",
    yref="y",
    text="Linha Neutra<br>(Mínimo Aceitável)",
    showarrow=True,
    font=dict(size=16, color="#000000"),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor='white',
    opacity=1
)

graph1.update_layout(legend_title_text='Legenda')
graph1.update_layout(hovermode='y unified')

# Gráfico 2: Scatter plot para a análise das cores (similar ao segundo exemplo)
df_cores = calculo_df[['Modelo', 'Cor']].copy()
df_cores['Valor X'] = 0
df_cores['Tam_scatter'] = 20
df_cores['Definicao'] = df_cores['Modelo'].apply(lambda x: 'Classificador' if x in ['BOM', 'NEUTRO'] else 'Desempenho')

# Adicionando valores de referência para cores
cores_valores = {
    'Branco': 90,
    'Vermelho': 80,
    'Azul': 70,
    'Preto': 60,
    'Prata': 50,
    'Cinza': 40,
    'Verde': 30
}

df_cores['Valor Cores'] = df_cores['Cor'].map(cores_valores)
df_cores.loc[df_cores['Modelo'] == 'BOM', 'Valor Cores'] = 80  # Valor BOM para cor
df_cores.loc[df_cores['Modelo'] == 'NEUTRO', 'Valor Cores'] = 50  # Valor NEUTRO para cor

graph2 = px.scatter(df_cores, x='Valor X', y='Valor Cores',
                   title='Análise de Cores dos Modelos',
                   color='Cor',
                   symbol='Definicao',
                   symbol_map={'Classificador': 'diamond-wide', 'Desempenho': 'circle'},
                   hover_name='Modelo',
                   size='Tam_scatter',
                   hover_data={'Cor': True, 'Valor Cores': True, 'Valor X': False})

graph2.update_traces(marker={'size': 20})
graph2.update_xaxes(fixedrange=True, title=None, showticklabels=False, showgrid=False)
graph2.update_yaxes(fixedrange=True, title=None, showticklabels=False, showgrid=False)

# Adicionando linhas de referência
graph2.add_hline(y=50, line_color='gray', line_width=3, line_dash='dash',
                annotation_text="NEUTRO", annotation_position="top right")
graph2.add_hline(y=80, line_color='red', line_width=3, line_dash='dash',
                annotation_text="BOM", annotation_position="top right")

graph2.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=1,
    title_text='Legenda'
))

graph2.add_vline(x=0, y0=0, y1=1)

# Exibindo os gráficos
graph1.show()
graph2.show()

# Identificando os produtos recomendados
produtos_desempenho = calculo_df[~calculo_df['Modelo'].isin(['BOM', 'NEUTRO'])]
if not produtos_desempenho.empty:
    pontuacao_maxima = produtos_desempenho['Pontuacao_Total'].max()
    produtos_recomendados = produtos_desempenho[produtos_desempenho['Pontuacao_Total'] == pontuacao_maxima]

    print("\nProdutos recomendados:")
    for _, produto in produtos_recomendados.iterrows():
        print(f"\nProduto: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
        print("Pontos de vantagem e justificativas:")

        # Adicionando análise detalhada para cada atributo
        for col in pesos.index:
            if tipos_dados[col] in ['number', 'boolean']:
                valor = produto[col]
                bom = valores_bom_df[col]
                neutro = valores_neutro_df[col]
                peso = pesos[col]
                prop = proporcionalidade[col]

                pontuacao = calcular_pontuacao(valor, bom, neutro, peso, prop)

                if pontuacao > 0:
                    if prop == 'proportional':
                        print(f"- {col}: Performance {pontuacao/peso*100:.1f}% melhor que o mínimo aceitável")
                    elif prop == 'i_proportional':
                        print(f"- {col}: Performance {pontuacao/peso*100:.1f}% melhor que o mínimo aceitável (inversamente proporcional)")
                    else:
                        print(f"- {col}: Atende ao critério desejado")
                elif pontuacao < 0:
                    if prop == 'proportional':
                        print(f"- {col}: Performance {-pontuacao/peso*100:.1f}% pior que o mínimo aceitável")
                    elif prop == 'i_proportional':
                        print(f"- {col}: Performance {-pontuacao/peso*100:.1f}% pior que o mínimo aceitável (inversamente proporcional)")
                    else:
                        print(f"- {col}: Não atende ao critério mínimo")

            elif tipos_dados[col] == 'string' and col == 'Cor':
                pontuacao = produto['Cor_pontuacao']
                if pontuacao > 0:
                    print(f"- Cor: {produto['Cor']} (preferência positiva)")
                elif pontuacao < 0:
                    print(f"- Cor: {produto['Cor']} (não é a preferida)")
else:
    print("Nenhum produto de desempenho encontrado para recomendação.")


Produtos recomendados:

Produto: Honda City com pontuação total de 0.53
Pontos de vantagem e justificativas:
- Preço (R$): Performance 8.2% pior que o mínimo aceitável (inversamente proporcional)
- Consumo (km/l): Performance 36.0% melhor que o mínimo aceitável
- Potência (cv): Performance 120.0% melhor que o mínimo aceitável
- Porta-malas (litros): Performance 200.0% melhor que o mínimo aceitável
- Segurança (estrelas): Performance 100.0% melhor que o mínimo aceitável
- Câmbio Automático: Atende ao critério desejado
- Cor: Azul (não é a preferida)


In [15]:
df_cores.head()

,Modelo,Cor,Valor X,Tam_scatter,Definicao,Valor Cores
0,BOM,Branco,0,20,Classificador,80
1,NEUTRO,Prata,0,20,Classificador,50
2,Fiat Argo,Vermelho,0,20,Desempenho,80
3,Volkswagen Polo,Azul,0,20,Desempenho,70
4,Chevrolet Onix,Preto,0,20,Desempenho,60


In [19]:
import pandas as pd
import plotly.express as px
import numpy as np

# Leitura do arquivo CSV
teste_df = pd.read_csv('base_carro.csv', encoding='UTF-8', sep=',')

# Extraindo os pesos e tipos de dados
pesos = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
proporcionalidade = teste_df.iloc[2].drop('Modelo')

# Separando os DataFrames de 'BOM', 'NEUTRO' e os demais para cálculo
valores_bom_df = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro_df = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')
calculo_df = teste_df.drop([0, 1, 2], errors='ignore').reset_index(drop=True)

# Convertendo os tipos de dados
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom_df[col] = float(valores_bom_df[col])
        valores_neutro_df[col] = float(valores_neutro_df[col])
        calculo_df[col] = calculo_df[col].astype(float)
    elif tipos_dados[col] == 'boolean':
        valores_bom_df[col] = valores_bom_df[col] == 'TRUE'
        valores_neutro_df[col] = valores_neutro_df[col] == 'TRUE'
        calculo_df[col] = calculo_df[col].map({'TRUE': True, 'FALSE': False})

# Função de pontuação para dados numéricos e booleanos
def calcular_pontuacao(valor, bom, neutro, peso, proporcionalidade_tipo):
    if bom == neutro:
        return 0
    elif proporcionalidade_tipo == 'proportional':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif bom == neutro:
            return 0
        elif valor >= neutro:
            return ((valor - neutro) / (bom - neutro)) * peso
        else:
            return -((neutro - valor) / neutro) * peso
    elif proporcionalidade_tipo == 'i_proportional':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif bom < neutro:
            if valor <= bom:
                return peso
            elif bom < valor <= neutro:
                return ((neutro - valor) / (neutro - bom)) * peso
            elif valor > neutro:
                return -((valor - neutro) / neutro) * peso
        elif bom > neutro:
            if valor >= bom:
                return peso
            elif neutro <= valor < bom:
                return ((valor - neutro) / (bom - neutro)) * peso
            elif valor < neutro:
                return -((neutro - valor) / neutro) * peso
        else:
            return 0
    elif proporcionalidade_tipo == 'boolean':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        else:
            return 0
    else:
        return 0

# Função de pontuação para dados de string (cor)
def calcular_pontuacao_cor(cor, peso):
    valores_cores = {
        'Branco': 0.05,
        'Vermelho': 0.01,
        'Azul': 0.02,
        'Preto': 0.01,
        'Prata': 0.04,
        'Cinza': 0.03
    }

    valor_bom = valores_bom_df['Cor']
    valor_neutro = valores_neutro_df['Cor']

    if cor == valor_bom:
        return peso * valores_cores.get(cor, 0.01)
    elif cor == valor_neutro:
        return 0
    else:
        return -peso * 0.01

# Aplicando a função de pontuação
for col in pesos.index:
    if tipos_dados[col] in ['number', 'boolean']:
        calculo_df[col + '_pontuacao'] = calculo_df.apply(
            lambda x: calcular_pontuacao(x[col], valores_bom_df[col], valores_neutro_df[col], pesos[col], proporcionalidade[col]),
            axis=1
        )
    elif col == 'Cor' and tipos_dados[col] == 'string':
        calculo_df['Cor_pontuacao'] = calculo_df.apply(
            lambda x: calcular_pontuacao_cor(x['Cor'], pesos['Cor']),
            axis=1
        )

# Somando as pontuações
colunas_pontuacao = [col + '_pontuacao' for col in pesos.index if tipos_dados[col] in ['number', 'boolean', 'string']]
calculo_df['Pontuacao_Total'] = calculo_df[colunas_pontuacao].sum(axis=1)

# 1. GRÁFICO DE BARRAS PARA PONTUAÇÃO TOTAL (NUMÉRICOS)
df_analise = calculo_df[['Modelo', 'Pontuacao_Total']].copy()

# Adicionando uma coluna para colorir os pontos no gráfico
df_analise['Color'] = df_analise['Modelo'].apply(
    lambda x: 'Classificadores' if x in ['BOM', 'NEUTRO'] else
    ('Pontuação Positiva' if x in df_analise['Modelo'] and df_analise.loc[df_analise['Modelo'] == x, 'Pontuacao_Total'].iloc[0] >= 0
     else 'Pontuação Negativa')
)

# Mapeamento de cores
color_mapping = {
    'Classificadores': 'Classificador',
    'Pontuação Positiva': 'Pontuação positiva',
    'Pontuação Negativa': 'Pontuação negativa'
}

df_analise['Color'] = df_analise['Color'].map(color_mapping)
df_analise_sorted = df_analise.sort_values(by='Pontuacao_Total', ascending=False, na_position='last').reset_index(drop=True)

# Gráfico 1: Barra com a pontuação total
graph1 = px.bar(df_analise_sorted, x='Pontuacao_Total', y='Modelo', orientation='h',
               color='Color', text_auto=False, hover_name='Modelo',
               hover_data={'Color': False, 'Modelo': False, 'Pontuacao_Total': True},
               color_discrete_sequence=["cornflowerblue", "lightgreen", "palevioletred"],
               title='Pontuação Total dos Modelos')

graph1.update_layout(yaxis={'categoryorder': 'array', 'categoryarray': df_analise_sorted['Modelo'].tolist()[::-1]})
graph1.update_xaxes(showticklabels=True)
graph1.update_yaxes(showticklabels=True)
graph1.update_coloraxes(showscale=False)
graph1.update_layout(xaxis_title='')

graph1.add_vline(x=0, line_dash='dash', line_color='black')
graph1.add_annotation(
    x=0,
    y=len(df_analise_sorted) - 1,
    xref="x",
    yref="y",
    text="Linha Neutra<br>(Mínimo Aceitável)",
    showarrow=True,
    font=dict(size=16, color="#000000"),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor='white',
    opacity=1
)

graph1.update_layout(legend_title_text='Legenda')
graph1.update_layout(hovermode='y unified')

# 2. GRÁFICO SCATTER PARA STRING (CORES)
df_cores = calculo_df[['Modelo', 'Cor']].copy()
df_cores['Valor X'] = 0
df_cores['Tam_scatter'] = 20
df_cores['Definicao'] = df_cores['Modelo'].apply(lambda x: 'Classificador' if x in ['BOM', 'NEUTRO'] else 'Desempenho')

# Adicionando valores de referência para cores
cores_valores = {
    'Branco': 90,
    'Vermelho': 80,
    'Azul': 70,
    'Preto': 60,
    'Prata': 50,
    'Cinza': 40,
    'Verde': 30
}

df_cores['Valor Cores'] = df_cores['Cor'].map(cores_valores)
df_cores.loc[df_cores['Modelo'] == 'BOM', 'Valor Cores'] = 80  # Valor BOM para cor
df_cores.loc[df_cores['Modelo'] == 'NEUTRO', 'Valor Cores'] = 50  # Valor NEUTRO para cor

graph2 = px.scatter(df_cores, x='Valor X', y='Valor Cores',
                   title='Análise de Cores dos Modelos',
                   color='Cor',
                   symbol='Definicao',
                   symbol_map={'Classificador': 'diamond-wide', 'Desempenho': 'circle'},
                   hover_name='Modelo',
                   size='Tam_scatter',
                   hover_data={'Cor': True, 'Valor Cores': True, 'Valor X': False})

graph2.update_traces(marker={'size': 20})
graph2.update_xaxes(fixedrange=True, title=None, showticklabels=False, showgrid=False)
graph2.update_yaxes(fixedrange=True, title=None, showticklabels=False, showgrid=False)

# Adicionando linhas de referência
graph2.add_hline(y=50, line_color='gray', line_width=3, line_dash='dash',
                annotation_text="NEUTRO", annotation_position="top right")
graph2.add_hline(y=80, line_color='red', line_width=3, line_dash='dash',
                annotation_text="BOM", annotation_position="top right")

graph2.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=1,
    title_text='Legenda'
))

graph2.add_vline(x=0, y0=0, y1=1)

# 3. GRÁFICO DE LINHAS PARA VALORES BOOLEANOS
# Preparando dados para booleanos
atributos_booleanos = [col for col in pesos.index if tipos_dados[col] == 'boolean']

for atributo in atributos_booleanos:
    df_boolean = calculo_df[['Modelo', atributo]].copy()
    df_boolean['Valor'] = df_boolean[atributo].map({True: 1, False: 0})
    df_boolean['Tipo'] = df_boolean['Modelo'].apply(lambda x: 'Classificador' if x in ['BOM', 'NEUTRO'] else 'Desempenho')

    # Adicionando linhas de referência
    valor_bom = 1 if valores_bom_df[atributo] else 0
    valor_neutro = 1 if valores_neutro_df[atributo] else 0

    graph3 = px.line(df_boolean, x='Modelo', y='Valor',
                    title=f'Análise de {atributo}',
                    markers=True,
                    color='Tipo',
                    color_discrete_map={'Classificador': 'cornflowerblue', 'Desempenho': 'lightgreen'})

    # Adicionando pontos para melhor visualização
    graph3.add_scatter(x=df_boolean['Modelo'], y=df_boolean['Valor'],
                      mode='markers', marker=dict(size=12),
                      showlegend=False)

    # Linhas de referência
    graph3.add_hline(y=valor_bom, line_color='blue', line_width=2, line_dash='dash',
                    annotation_text="BOM", annotation_position="top right")
    graph3.add_hline(y=valor_neutro, line_color='gray', line_width=2, line_dash='dash',
                    annotation_text="NEUTRO", annotation_position="top right")

    graph3.update_yaxes(range=[-0.1, 1.1], tickvals=[0, 1], ticktext=['FALSE', 'TRUE'])
    graph3.update_layout(xaxis_title='Modelo', yaxis_title='Valor')

    # Exibindo os gráficos
    graph1.show()
    graph2.show()
    graph3.show()

# Identificando os produtos recomendados
produtos_desempenho = calculo_df[~calculo_df['Modelo'].isin(['BOM', 'NEUTRO'])]
if not produtos_desempenho.empty:
    pontuacao_maxima = produtos_desempenho['Pontuacao_Total'].max()
    produtos_recomendados = produtos_desempenho[produtos_desempenho['Pontuacao_Total'] == pontuacao_maxima]

    print("\nProdutos recomendados:")
    for _, produto in produtos_recomendados.iterrows():
        print(f"\nProduto: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
        print("Pontos de vantagem e justificativas:")

        for col in pesos.index:
            if tipos_dados[col] in ['number', 'boolean']:
                valor = produto[col]
                bom = valores_bom_df[col]
                neutro = valores_neutro_df[col]
                peso = pesos[col]
                prop = proporcionalidade[col]

                pontuacao = produto[col + '_pontuacao']

                if pontuacao > 0:
                    if prop == 'proportional':
                        print(f"- {col}: Performance {pontuacao/peso*100:.1f}% melhor que o mínimo aceitável")
                    elif prop == 'i_proportional':
                        print(f"- {col}: Performance {pontuacao/peso*100:.1f}% melhor que o mínimo aceitável (inversamente proporcional)")
                    else:
                        print(f"- {col}: Atende ao critério desejado")
                elif pontuacao < 0:
                    if prop == 'proportional':
                        print(f"- {col}: Performance {-pontuacao/peso*100:.1f}% pior que o mínimo aceitável")
                    elif prop == 'i_proportional':
                        print(f"- {col}: Performance {-pontuacao/peso*100:.1f}% pior que o mínimo aceitável (inversamente proporcional)")
                    else:
                        print(f"- {col}: Não atende ao critério mínimo")

            elif tipos_dados[col] == 'string' and col == 'Cor':
                pontuacao = produto['Cor_pontuacao']
                if pontuacao > 0:
                    print(f"- Cor: {produto['Cor']} (preferência positiva)")
                elif pontuacao < 0:
                    print(f"- Cor: {produto['Cor']} (não é a preferida)")
else:
    print("Nenhum produto de desempenho encontrado para recomendação.")


Produtos recomendados:

Produto: Honda City com pontuação total de 0.53
Pontos de vantagem e justificativas:
- Preço (R$): Performance 8.2% pior que o mínimo aceitável (inversamente proporcional)
- Consumo (km/l): Performance 36.0% melhor que o mínimo aceitável
- Potência (cv): Performance 120.0% melhor que o mínimo aceitável
- Porta-malas (litros): Performance 200.0% melhor que o mínimo aceitável
- Segurança (estrelas): Performance 100.0% melhor que o mínimo aceitável
- Câmbio Automático: Atende ao critério desejado
- Cor: Azul (não é a preferida)


In [24]:
# Preparando os dados
df_scatter = calculo_df[['Modelo', 'Câmbio Automático']].copy()
df_scatter['Valor'] = df_scatter['Câmbio Automático'].map({True: 1, False: 0})
df_scatter['Tamanho'] = 20

# Criando o gráfico
fig = px.scatter(df_scatter,
                 x='Modelo',
                 y='Valor',
                 color='Câmbio Automático',
                 color_discrete_map={True: '#4CAF50', False: '#F44336'},
                 size='Tamanho',
                 title='Câmbio Automático por Modelo',
                 hover_name='Modelo')

# Personalização
fig.update_yaxes(range=[-0.5, 1.5],
                tickvals=[0, 1],
                ticktext=['Manual', 'Automático'])
fig.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')))
fig.update_layout(xaxis_title='', yaxis_title='')

# Adicionando referências
fig.add_hline(y=1 if valores_bom_df['Câmbio Automático'] else 0,
              line_dash='dot',
              line_color='blue',
              annotation_text="BOM",
              annotation_position="top right")

fig.add_hline(y=1 if valores_neutro_df['Câmbio Automático'] else 0,
              line_dash='dash',
              line_color='gray',
              annotation_text="NEUTRO",
              annotation_position="bottom right")

fig.show()

In [71]:
import pandas as pd
import plotly.express as px
import numpy as np

# Leitura do arquivo CSV
teste_df = pd.read_csv('base_carro.csv', encoding='UTF-8', sep=',')

# Extraindo os pesos e tipos de dados
pesos = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
proporcionalidade = teste_df.iloc[2].drop('Modelo')

# Separando os DataFrames de 'BOM', 'NEUTRO' e os demais para cálculo
valores_bom_df = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro_df = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')
calculo_df = teste_df.drop([0, 1, 2], errors='ignore').reset_index(drop=True)

# Convertendo os tipos de dados
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom_df[col] = float(valores_bom_df[col])
        valores_neutro_df[col] = float(valores_neutro_df[col])
        calculo_df[col] = calculo_df[col].astype(float)
    elif tipos_dados[col] == 'boolean':
        valores_bom_df[col] = valores_bom_df[col] == 'TRUE'
        valores_neutro_df[col] = valores_neutro_df[col] == 'TRUE'
        calculo_df[col] = calculo_df[col].map({'TRUE': True, 'FALSE': False})

# Função de pontuação para dados numéricos e booleanos
def calcular_pontuacao(valor, bom, neutro, peso, proporcionalidade_tipo):
    if bom == neutro:
        return 0
    elif proporcionalidade_tipo == 'proportional':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif bom == neutro:
            return 0
        elif valor >= neutro:
            return ((valor - neutro) / (bom - neutro)) * peso
        else:
            return -((neutro - valor) / neutro) * peso
    elif proporcionalidade_tipo == 'i_proportional':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif bom < neutro:
            if valor <= bom:
                return peso
            elif bom < valor <= neutro:
                return ((neutro - valor) / (neutro - bom)) * peso
            elif valor > neutro:
                return -((valor - neutro) / neutro) * peso
        elif bom > neutro:
            if valor >= bom:
                return peso
            elif neutro <= valor < bom:
                return ((valor - neutro) / (bom - neutro)) * peso
            elif valor < neutro:
                return -((neutro - valor) / neutro) * peso
        else:
            return 0
    elif proporcionalidade_tipo == 'boolean':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        else:
            return 0
    else:
        return 0

# Função de pontuação para dados de string (cor)
def calcular_pontuacao_cor(cor, peso):
    valores_cores = {
        'Branco': 0.05,
        'Vermelho': 0.01,
        'Azul': 0.02,
        'Preto': 0.01,
        'Prata': 0.04,
        'Cinza': 0.03
    }

    valor_bom = valores_bom_df['Cor']
    valor_neutro = valores_neutro_df['Cor']

    if cor == valor_bom:
        return peso * valores_cores.get(cor, 0.01)
    elif cor == valor_neutro:
        return 0
    else:
        return -peso * 0.01

# Aplicando a função de pontuação
for col in pesos.index:
    if tipos_dados[col] in ['number', 'boolean']:
        calculo_df[col + '_pontuacao'] = calculo_df.apply(
            lambda x: calcular_pontuacao(x[col], valores_bom_df[col], valores_neutro_df[col], pesos[col], proporcionalidade[col]),
            axis=1
        )
    elif col == 'Cor' and tipos_dados[col] == 'string':
        calculo_df['Cor_pontuacao'] = calculo_df.apply(
            lambda x: calcular_pontuacao_cor(x['Cor'], pesos['Cor']),
            axis=1
        )

# Somando as pontuações
colunas_pontuacao = [col + '_pontuacao' for col in pesos.index if tipos_dados[col] in ['number', 'boolean', 'string']]
calculo_df['Pontuacao_Total'] = calculo_df[colunas_pontuacao].sum(axis=1)

# 1. GRÁFICO DE BARRAS PARA PONTUAÇÃO TOTAL (NUMÉRICOS)
df_analise = calculo_df[['Modelo', 'Pontuacao_Total']].copy()

# Correção crucial: simplificar a lógica de classificação
df_analise['Color_Category'] = df_analise.apply(
    lambda x: 'Classificador' if x['Modelo'] in ['BOM', 'NEUTRO'] else
    ('Positiva' if x['Pontuacao_Total'] >= 0 else 'Negativa'),
    axis=1
)

# Mapeamento de cores CORRETO e consistente
color_map = {
    'Classificador': 'cornflowerblue',
    'Positiva': 'lightgreen',  # Verde para positivos
    'Negativa': 'palevioletred'  # Vermelho para negativos
}

# Ordem explícita para a legenda
category_order = ['Classificador', 'Positiva', 'Negativa']

df_analise_sorted = df_analise.sort_values(by='Pontuacao_Total', ascending=False, na_position='last').reset_index(drop=True)

# Criar o gráfico com configuração explícita
graph1 = px.bar(df_analise_sorted,
               x='Pontuacao_Total',
               y='Modelo',
               orientation='h',
               color='Color_Category',
               color_discrete_map=color_map,
               category_orders={'Color_Category': category_order},
               hover_name='Modelo',
               title='Pontuação Total dos Modelos',
               labels={'Color_Category': 'Legenda'})

# Configuração do layout
graph1.update_layout(
    yaxis={'categoryorder': 'array', 'categoryarray': df_analise_sorted['Modelo'].tolist()[::-1]},
    xaxis={'showticklabels': True, 'showgrid': True, 'gridcolor': 'lightgray'},
    yaxis_showgrid=True,
    yaxis_gridcolor='lightgray',
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title_text='',
    legend_traceorder='normal'
)

# Linha neutra e demais elementos
graph1.add_vline(x=0, line_dash='dash', line_color='black')
graph1.update_xaxes(title_text="Pontuação Total")
graph1.update_layout(hovermode='y unified')

# Correção final da legenda
graph1.for_each_trace(lambda t: t.update(name='Pontuação ' + t.name))

# 2. GRÁFICO SCATTER PARA STRINGS

df_cores['Valor Cores'] = df_cores['Cor'].map(cores_valores)

graph2 = px.scatter(df_cores, x='Valor X', y='Valor Cores',
                   title='Análise de Cores dos Modelos',
                   color='Cor',
                   hover_name='Modelo',
                   hover_data={'Cor': True, 'Valor Cores': True, 'Valor X': False})

graph2.update_traces(
    marker={
        'size': 20,
        'line': {'width': 1, 'color': 'DarkSlateGrey'},
        'opacity': 0.8  # Ajuste para melhor visibilidade sobre o fundo
    }
)

graph2.update_layout(
    xaxis={
        'fixedrange': True,
        'showticklabels': False,  # Remove os números do eixo X
        'showgrid': False,  # Remove linhas de grade verticais
        'zeroline': False  # Remove a linha do zero
    },
    yaxis={
        'fixedrange': True,
        'showticklabels': False,  # Remove os números do eixo Y
        'showgrid': False,  # Remove linhas de grade horizontais
        'zeroline': False  # Remove a linha do zero
    },
    paper_bgcolor='white',
    plot_bgcolor='white',
    margin=dict(l=20, r=20, t=40, b=20)  # Ajuste das margens
)

# Linhas de referência com estilo mais discreto
graph2.add_hline(
    y=50,
    line_color='gray',
    line_width=1.5,
    line_dash='solid',
    annotation_text="NEUTRO",
    annotation_position="top right",
    annotation_font_size=12
)

graph2.add_hline(
    y=80,
    line_color='gray',
    line_width=1.5,
    line_dash='solid',
    annotation_text="BOM",
    annotation_position="top right",
    annotation_font_size=12
)

# Configuração da legenda
graph2.update_layout(
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.2,
        xanchor='right',
        x=1,
        title_text='Legenda',
        bgcolor='rgba(255, 255, 255, 0.8)'  # Fundo semi-transparente
    )
)

# Ajuste para melhorar a visualização dos pontos
graph2.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
        font_family="Arial"
    )
)

# 3. GRÁFICO BOOLEANO (CÂMBIO AUTOMÁTICO)
df_boolean = calculo_df[~calculo_df['Modelo'].isin(['BOM', 'NEUTRO'])][['Modelo', 'Câmbio Automático']].copy()
df_boolean['Valor'] = df_boolean['Câmbio Automático'].map({True: 1, False: 0})
df_boolean['Tamanho'] = 20

# Criando o gráfico
graph3 = px.scatter(df_boolean,
                   x='Modelo',
                   y='Valor',
                   color='Câmbio Automático',
                   color_discrete_map={True: '#4CAF50', False: '#F44336'},
                   size='Tamanho',
                   title='Câmbio Automático por Modelo',
                   hover_name='Modelo')

# Personalização do eixo Y (rótulos SIM/NÃO)
graph3.update_yaxes(
    range=[-0.5, 1.5],
    tickvals=[0, 1],
    ticktext=['Não', 'Sim'],  # Alterado para português
    showgrid=False
)

# Atualizando a LEGENDA para mostrar "Sim" e "Não"
graph3.update_layout(
    legend_title_text='Câmbio Automático',
    legend=dict(
        itemsizing='constant',
        title_font=dict(size=12),
        traceorder='normal',
        itemclick=False,
        itemdoubleclick=False
    )
)

# Sobrescrevendo os rótulos da legenda
graph3.for_each_trace(lambda t: t.update(name='Sim' if t.name == 'True' else 'Não'))

# Restante das personalizações (mantido igual)
graph3.update_xaxes(showgrid=False)
graph3.update_traces(marker=dict(line=dict(width=1.5, color='DarkSlateGrey')))
graph3.update_layout(
    xaxis_title='',
    yaxis_title='',
    plot_bgcolor='white'
)

# Linhas de referência BOM/NEUTRO (com texto em português)
valor_bom = 1 if valores_bom_df['Câmbio Automático'] else 0
valor_neutro = 1 if valores_neutro_df['Câmbio Automático'] else 0

graph3.add_hline(y=valor_bom,
                line_dash='dash',
                line_color='gray',
                line_width=2,
                 annotation_text=f"BOM",
                annotation_position="top right")

graph3.add_hline(y=valor_neutro,
                line_dash='dash',
                line_color='gray',
                line_width=2,
                annotation_text=f"NEUTRO",
                annotation_position="bottom right")

# Exibindo os gráficos
graph1.show()
graph2.show()
graph3.show()

# Identificando os produtos recomendados
produtos_desempenho = calculo_df[~calculo_df['Modelo'].isin(['BOM', 'NEUTRO'])]
if not produtos_desempenho.empty:
    pontuacao_maxima = produtos_desempenho['Pontuacao_Total'].max()
    produtos_recomendados = produtos_desempenho[produtos_desempenho['Pontuacao_Total'] == pontuacao_maxima]

    print("\nProdutos recomendados:")
    for _, produto in produtos_recomendados.iterrows():
        print(f"\nProduto: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
        print("Pontos de vantagem e justificativas:")

        for col in pesos.index:
            if tipos_dados[col] in ['number', 'boolean']:
                valor = produto[col]
                bom = valores_bom_df[col]
                neutro = valores_neutro_df[col]
                peso = pesos[col]
                prop = proporcionalidade[col]

                pontuacao = produto[col + '_pontuacao']

                if pontuacao > 0:
                    if prop == 'proportional':
                        print(f"- {col}: Performance {pontuacao/peso*100:.1f}% melhor que o mínimo aceitável")
                    elif prop == 'i_proportional':
                        print(f"- {col}: Performance {pontuacao/peso*100:.1f}% melhor que o mínimo aceitável (inversamente proporcional)")
                    else:
                        print(f"- {col}: Atende ao critério desejado")
                elif pontuacao < 0:
                    if prop == 'proportional':
                        print(f"- {col}: Performance {-pontuacao/peso*100:.1f}% pior que o mínimo aceitável")
                    elif prop == 'i_proportional':
                        print(f"- {col}: Performance {-pontuacao/peso*100:.1f}% pior que o mínimo aceitável (inversamente proporcional)")
                    else:
                        print(f"- {col}: Não atende ao critério mínimo")

            elif tipos_dados[col] == 'string' and col == 'Cor':
                pontuacao = produto['Cor_pontuacao']
                if pontuacao > 0:
                    print(f"- Cor: {produto['Cor']} (preferência positiva)")
                elif pontuacao < 0:
                    print(f"- Cor: {produto['Cor']} (não é a preferida)")
else:
    print("Nenhum produto de desempenho encontrado para recomendação.")


Produtos recomendados:

Produto: Honda City com pontuação total de 0.53
Pontos de vantagem e justificativas:
- Preço (R$): Performance 8.2% pior que o mínimo aceitável (inversamente proporcional)
- Consumo (km/l): Performance 36.0% melhor que o mínimo aceitável
- Potência (cv): Performance 120.0% melhor que o mínimo aceitável
- Porta-malas (litros): Performance 200.0% melhor que o mínimo aceitável
- Segurança (estrelas): Performance 100.0% melhor que o mínimo aceitável
- Câmbio Automático: Atende ao critério desejado
- Cor: Azul (não é a preferida)


In [30]:
import pandas as pd
import plotly.express as px
import numpy as np

class DataAnalyzer:
    def __init__(self, file_path):
        self.file_path = file_path
        self.df = None
        self.pesos = None
        self.tipos_dados = None
        self.proporcionalidade = None
        self.valores_bom = None
        self.valores_neutro = None
        self.calculo_df = None

    def load_and_prepare_data(self):
        """Carrega e prepara os dados"""
        self.df = pd.read_csv(self.file_path, encoding='UTF-8', sep=',')

        # Extrai pesos, tipos e proporcionalidade
        self.pesos = self.df.iloc[0].dropna().astype(float)
        self.tipos_dados = self.df.iloc[1].dropna()
        self.proporcionalidade = self.df.iloc[2].dropna()

        # Extrai valores BOM e NEUTRO
        self.valores_bom = self.df[self.df['Modelo'] == 'BOM'].iloc[0].dropna()
        self.valores_neutro = self.df[self.df['Modelo'] == 'NEUTRO'].iloc[0].dropna()

        # DataFrame para cálculo (exclui linhas de configuração)
        self.calculo_df = self.df.drop([0, 1, 2], errors='ignore').reset_index(drop=True)

        # Converte tipos de dados
        self._convert_data_types()

    def _convert_data_types(self):
        """Converte os tipos de dados conforme especificado"""
        for col in self.tipos_dados.index:
            if col not in self.calculo_df.columns:
                continue

            if self.tipos_dados[col] == 'number':
                self.valores_bom[col] = float(self.valores_bom[col])
                self.valores_neutro[col] = float(self.valores_neutro[col])
                self.calculo_df[col] = pd.to_numeric(self.calculo_df[col], errors='coerce')
            elif self.tipos_dados[col] == 'boolean':
                self.valores_bom[col] = self.valores_bom[col] == 'TRUE'
                self.valores_neutro[col] = self.valores_neutro[col] == 'FALSE'
                self.calculo_df[col] = self.calculo_df[col].map({'TRUE': True, 'FALSE': False, True: True, False: False})
            elif self.tipos_dados[col] == 'string':
                self.calculo_df[col] = self.calculo_df[col].astype(str)

    def calcular_pontuacao_numerica(self, valor, bom, neutro, peso, proporcionalidade_tipo):
        """Calcula pontuação para dados numéricos e booleanos"""
        if bom == neutro:
            return 0

        if proporcionalidade_tipo == 'proportional':
            if isinstance(valor, bool):
                return peso if valor == bom else (-peso if valor != neutro else 0)
            return ((valor - neutro) / (bom - neutro)) * peso if valor >= neutro else -((neutro - valor) / neutro) * peso

        elif proporcionalidade_tipo == 'i_proportional':
            if isinstance(valor, bool):
                return peso if valor == bom else (-peso if valor != neutro else 0)
            if bom < neutro:
                if valor <= bom: return peso
                elif valor <= neutro: return ((neutro - valor) / (neutro - bom)) * peso
                else: return -((valor - neutro) / neutro) * peso
            else:
                if valor >= bom: return peso
                elif valor >= neutro: return ((valor - neutro) / (bom - neutro)) * peso
                else: return -((neutro - valor) / neutro) * peso

        elif proporcionalidade_tipo == 'boolean':
            return peso if valor == bom else (-peso if valor != neutro else 0)

        return 0

    def calcular_pontuacao_string(self, valor, coluna):
        """Calcula pontuação para dados de string de forma genérica"""
        peso = self.pesos[coluna]
        valor_bom = self.valores_bom[coluna]
        valor_neutro = self.valores_neutro[coluna]

        # Verifica se há valores específicos para strings (como cores)
        if f"{coluna},pts_string" in self.df.columns:
            valores_string = dict(zip(
                self.df[coluna].dropna(),
                self.df[f"{coluna},pts_string"].dropna().astype(float)
            ))
            if valor in valores_string:
                if valor == valor_bom:
                    return peso * valores_string[valor]
                elif valor == valor_neutro:
                    return 0
                else:
                    return -peso * 0.01  # Penalização padrão para valores não preferidos

        # Lógica genérica para strings sem valores específicos
        if valor == valor_bom:
            return peso
        elif valor == valor_neutro:
            return 0
        return -peso * 0.1

    def calcular_pontuacoes(self):
        """Calcula todas as pontuações"""
        for col in self.pesos.index:
            if col not in self.calculo_df.columns:
                continue

            if self.tipos_dados[col] in ['number', 'boolean']:
                self.calculo_df[f"{col}_pontuacao"] = self.calculo_df.apply(
                    lambda x: self.calcular_pontuacao_numerica(
                        x[col],
                        self.valores_bom[col],
                        self.valores_neutro[col],
                        self.pesos[col],
                        self.proporcionalidade[col]
                    ), axis=1
                )
            elif self.tipos_dados[col] == 'string':
                self.calculo_df[f"{col}_pontuacao"] = self.calculo_df.apply(
                    lambda x: self.calcular_pontuacao_string(x[col], col),
                    axis=1
                )

        # Calcula pontuação total
        colunas_pontuacao = [col for col in self.calculo_df.columns if '_pontuacao' in col]
        self.calculo_df['Pontuacao_Total'] = self.calculo_df[colunas_pontuacao].sum(axis=1)

    def mostrar_pontuacoes(self, apenas_pontuacoes=True):
      """Exibe o DataFrame com as pontuações calculadas

      Args:
          apenas_pontuacoes (bool): Se True, mostra apenas colunas de pontuação
                                    Se False, mostra todas as colunas
      """
      if apenas_pontuacoes:
          # Filtra apenas colunas que terminam com '_pontuacao' ou são 'Modelo' ou 'Pontuacao_Total'
          cols = [col for col in self.calculo_df.columns
                if col.endswith('_pontuacao') or col in ['Modelo', 'Pontuacao_Total']]
          display(self.calculo_df[cols])
      else:
          display(self.calculo_df)

    def gerar_graficos(self):
        """Gera os gráficos de análise"""
        self._gerar_grafico_barras()
        self._gerar_grafico_strings()
        self._gerar_grafico_booleanos()

    def _gerar_grafico_barras(self):
        """Gráfico de barras para pontuação total"""
        df_analise = self.calculo_df[['Modelo', 'Pontuacao_Total']].copy()

        df_analise['Color_Category'] = df_analise.apply(
            lambda x: 'Classificador' if x['Modelo'] in ['BOM', 'NEUTRO'] else
            ('Positiva' if x['Pontuacao_Total'] >= 0 else 'Negativa'),
            axis=1
        )

        color_map = {
            'Classificador': 'cornflowerblue',
            'Positiva': 'lightgreen',
            'Negativa': 'palevioletred'
        }

        df_analise_sorted = df_analise.sort_values('Pontuacao_Total', ascending=False)

        fig = px.bar(df_analise_sorted,
                    x='Pontuacao_Total',
                    y='Modelo',
                    orientation='h',
                    color='Color_Category',
                    color_discrete_map=color_map,
                    hover_name='Modelo',
                    title='Pontuação Total dos Modelos')

        fig.update_layout(
            yaxis={'categoryorder': 'array', 'categoryarray': df_analise_sorted['Modelo'].tolist()[::-1]},
            plot_bgcolor='white',
            paper_bgcolor='white',
            legend_title_text='Legenda',
            legend_traceorder='normal',
            xaxis_title='Pontuação Total',
            yaxis_title='Modelo'
        )

        fig.add_annotation(
            x=0,
            y=len(df_analise_sorted) - 1,
            xref="x",
            yref="y",
            text="Linha Neutra<br>(Mínimo Aceitável)",
            showarrow=True,
            font=dict(size=16, color="#000000"),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=0,
            ay=-45,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor='white',
            opacity=1
        )


        fig.add_vline(x=0, line_dash='dash', line_color='black')
        fig.show()

    def _gerar_grafico_strings(self):
        """Gráfico para variáveis de string"""
        # Identifica colunas de string
        string_cols = [col for col in self.tipos_dados.index
                      if self.tipos_dados[col] == 'string' and col in self.calculo_df.columns]

        if not string_cols:
            return

        # Pega a primeira coluna de string para o gráfico
        col_string = string_cols[0]

        # Cria DataFrame para o gráfico
        df_string = self.calculo_df[['Modelo', col_string]].copy()
        df_string = df_string[~df_string['Modelo'].isin(['BOM', 'NEUTRO'])]

        # Adiciona valores para plotagem
        if f"{col_string},pts_string" in self.df.columns:
            valores_string = dict(zip(
                self.df[col_string].dropna(),
                self.df[f"{col_string},pts_string"].dropna().astype(float)
            ))
            df_string['Valor'] = df_string[col_string].map(valores_string)
        else:
            # Valores padrão se não houver especificação
            unique_vals = df_string[col_string].unique()
            df_string['Valor'] = df_string[col_string].apply(lambda x: list(unique_vals).index(x) * 10)

        # Posição X aleatória para dispersão
        np.random.seed(42)
        df_string['Valor X'] = np.random.normal(0, 1, len(df_string))

        fig = px.scatter(df_string,
                x='Valor X',
                y='Valor',
                color=col_string,
                hover_name='Modelo',
                title=f'Análise de {col_string.capitalize()}',
                size_max=30,  # Aumenta o tamanho máximo das bolinhas
                hover_data={'Valor X': False, 'Valor': False})  # Remove VALOR do hover

        # Linhas de referência
        if f"{col_string},pts_string" in self.df.columns:
            valor_bom = valores_string.get(self.valores_bom[col_string], 0)
            valor_neutro = valores_string.get(self.valores_neutro[col_string], 0)
        else:
            valor_bom = 50  # Valor padrão
            valor_neutro = 25  # Valor padrão

        fig.add_hline(y=valor_bom, line_dash='dash', annotation_text="BOM")
        fig.add_hline(y=valor_neutro, line_dash='dash', annotation_text="NEUTRO")

        fig.update_layout(
            xaxis_title=None,
            xaxis=dict(
                showticklabels=False,
                showgrid=False
            ),
            yaxis_title=None,
            yaxis=dict(
                showticklabels=False,
                showgrid=False
            ),
            plot_bgcolor='white',
            paper_bgcolor='white'
        )

        fig.update_traces(marker=dict(size=10))

        fig.show()

    def _gerar_grafico_booleanos(self):
        """Gráfico para variáveis booleanas"""
        # Identifica colunas booleanas
        bool_cols = [col for col in self.tipos_dados.index
                    if self.tipos_dados[col] == 'boolean' and col in self.calculo_df.columns]

        if not bool_cols:
            return

        # Pega a primeira coluna booleana para o gráfico
        col_bool = bool_cols[0]

        df_bool = self.calculo_df[~self.calculo_df['Modelo'].isin(['BOM', 'NEUTRO'])][['Modelo', col_bool]].copy()
        df_bool['Valor'] = df_bool[col_bool].map({True: 1, False: 0})
        df_bool['Tamanho'] = 20

        fig = px.scatter(df_bool,
                x='Modelo',
                y='Valor',
                color=col_bool,
                color_discrete_map={True: '#4CAF50', False: '#F44336'},
                size='Tamanho',
                title=f'{col_bool} por Modelo',
                hover_name='Modelo',
                hover_data={'Valor': False, 'Tamanho': False},
                category_orders={col_bool: [True, False]})

        # Atualiza a legenda para mostrar SIM/NÃO
        fig.for_each_trace(lambda t: t.update(name='SIM' if t.name == 'True' else 'NÃO'))

        fig.update_yaxes(
            range=[-0.5, 1.5],
            tickvals=[0, 1],
            ticktext=['Não', 'Sim'],
            showgrid=False
        )

        fig.update_layout(
            xaxis_title=None,
            yaxis_title=None,
            plot_bgcolor='white',
            paper_bgcolor='white'
        )

        # Linhas de referência
        valor_bom = 1 if self.valores_bom[col_bool] else 0
        valor_neutro = 1 if self.valores_neutro[col_bool] else 0

        # Verifica se BOM e NEUTRO são iguais
        if valor_bom == valor_neutro:
            fig.add_hline(y=valor_bom,
                        line_dash='dash',
                        line_color='gray',
                        annotation_text="BOM/NEUTRO",
                        annotation_position="top right")
        else:
            fig.add_hline(y=valor_bom,
                        line_dash='dash',
                        line_color='gray',
                        annotation_text="BOM",
                        annotation_position="top right")
            fig.add_hline(y=valor_neutro,
                          line_dash='dash',
                          line_color='gray',
                          annotation_text="NEUTRO",
                          annotation_position="bottom right")

        fig.show()

    def recomendar_produtos(self):
        """Recomenda produtos com base na pontuação"""
        produtos = self.calculo_df[~self.calculo_df['Modelo'].isin(['BOM', 'NEUTRO'])]

        if produtos.empty:
            print("Nenhum produto encontrado para recomendação.")
            return

        pontuacao_maxima = produtos['Pontuacao_Total'].max()
        recomendados = produtos[produtos['Pontuacao_Total'] == pontuacao_maxima]

        print("\nProdutos recomendados:")
        for _, produto in recomendados.iterrows():
            print(f"\nProduto: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
            print("Pontos de vantagem e justificativas:")

            for col in self.pesos.index:
                if col not in self.calculo_df.columns:
                    continue

                pont_col = f"{col}_pontuacao"
                if pont_col not in produto:
                    continue

                pontuacao = produto[pont_col]

                if pontuacao > 0:
                    if self.tipos_dados[col] in ['number', 'boolean']:
                        prop = self.proporcionalidade[col]
                        peso = self.pesos[col]

                        if prop == 'proportional':
                            print(f"- {col}: Performance {pontuacao/peso*100:.1f}% melhor que o mínimo aceitável")
                        elif prop == 'i_proportional':
                            print(f"- {col}: Performance {pontuacao/peso*100:.1f}% melhor que o mínimo aceitável (inversamente proporcional)")
                        else:
                            print(f"- {col}: Atende ao critério desejado")

                    elif self.tipos_dados[col] == 'string':
                        print(f"- {col}: Valor '{produto[col]}' é preferido")

                elif pontuacao < 0:
                    if self.tipos_dados[col] in ['number', 'boolean']:
                        prop = self.proporcionalidade[col]
                        peso = self.pesos[col]

                        if prop == 'proportional':
                            print(f"- {col}: Performance {-pontuacao/peso*100:.1f}% pior que o mínimo aceitável")
                        elif prop == 'i_proportional':
                            print(f"- {col}: Performance {-pontuacao/peso*100:.1f}% pior que o mínimo aceitável (inversamente proporcional)")
                        else:
                            print(f"- {col}: Não atende ao critério mínimo")

                    elif self.tipos_dados[col] == 'string':
                        print(f"- {col}: Valor '{produto[col]}' não é o preferido")

# Exemplo de uso para a base de carros
print("Análise para base de carros:")
analyzer_carros = DataAnalyzer('base_carro.csv')
analyzer_carros.load_and_prepare_data()
analyzer_carros.calcular_pontuacoes()
analyzer_carros.gerar_graficos()
analyzer_carros.recomendar_produtos()

# Exemplo de uso para a base de eletrônicos
print("\nAnálise para base de eletrônicos:")
analyzer_eletronicos = DataAnalyzer('base_eletronicos.csv')
analyzer_eletronicos.load_and_prepare_data()
analyzer_eletronicos.calcular_pontuacoes()
analyzer_eletronicos.gerar_graficos()
analyzer_eletronicos.recomendar_produtos()

Análise para base de carros:



Produtos recomendados:

Produto: Honda City com pontuação total de 0.53
Pontos de vantagem e justificativas:
- Preço (R$): Performance 8.2% pior que o mínimo aceitável (inversamente proporcional)
- Consumo (km/l): Performance 36.0% melhor que o mínimo aceitável
- Potência (cv): Performance 120.0% melhor que o mínimo aceitável
- Porta-malas (litros): Performance 200.0% melhor que o mínimo aceitável
- Segurança (estrelas): Performance 100.0% melhor que o mínimo aceitável
- Ar Condicionado: Atende ao critério desejado
- Cor: Valor 'Azul' não é o preferido

Análise para base de eletrônicos:



Produtos recomendados:

Produto: Apple com pontuação total de 1.40
Pontos de vantagem e justificativas:
- Memória RAM: Performance 300.0% melhor que o mínimo aceitável
- Armazenamento: Performance 104.8% melhor que o mínimo aceitável
- Bateria mAh: Performance 150.0% melhor que o mínimo aceitável
- Cor: Valor 'Cinza' não é o preferido


In [31]:
analyzer_carros.mostrar_pontuacoes(apenas_pontuacoes=True)

,Modelo,Preço (R$)_pontuacao,Consumo (km/l)_pontuacao,Potência (cv)_pontuacao,Porta-malas (litros)_pontuacao,Segurança (estrelas)_pontuacao,Ar Condicionado_pontuacao,Câmbio Automático_pontuacao,Cor_pontuacao,Pontuacao_Total
0,BOM,0.300000,0.100,0.100000,0.050000,0.200000,0.1,0,0.050,0.900000
1,NEUTRO,0.000000,0.000,0.000000,0.000000,0.000000,0.1,0,0.000,0.100000
2,Fiat Argo,0.120000,0.070,0.063333,0.025000,0.100000,0.1,0,-0.005,0.473333
3,Volkswagen Polo,0.036000,0.056,0.086667,0.010000,0.200000,0.1,0,-0.005,0.483667
4,Chevrolet Onix,0.084000,0.084,0.053333,0.002500,0.100000,0.1,0,-0.005,0.418833
5,Hyundai HB20,0.156000,0.076,0.036667,0.000000,0.100000,0.1,0,0.050,0.518667
6,Renault Kwid,0.240000,0.110,-0.022222,-0.001667,0.000000,0.1,0,0.000,0.426111
7,Toyota Yaris,-0.010588,0.050,0.066667,0.005000,0.100000,0.1,0,-0.005,0.306078
8,Honda City,-0.024706,0.036,0.120000,0.100000,0.200000,0.1,0,-0.005,0.526294
9,Peugeot 208,0.060000,0.060,0.076667,0.007500,0.100000,0.1,0,-0.005,0.399167


In [89]:
import pandas as pd
import plotly.express as px
import numpy as np

class DataAnalyzer:
    def __init__(self, file_path):
        self.file_path = file_path
        self.df = None
        self.pesos = None
        self.tipos_dados = None
        self.proporcionalidade = None
        self.valores_bom = None
        self.valores_neutro = None
        self.calculo_df = None

    def load_and_prepare_data(self):
        """Carrega e prepara os dados"""
        self.df = pd.read_csv(self.file_path, encoding='UTF-8', sep=',')

        # Extrai pesos, tipos e proporcionalidade
        self.pesos = self.df.iloc[0].dropna().astype(float)
        self.tipos_dados = self.df.iloc[1].dropna()
        self.proporcionalidade = self.df.iloc[2].dropna()

        # Extrai valores BOM e NEUTRO
        self.valores_bom = self.df[self.df['Modelo'] == 'BOM'].iloc[0].dropna()
        self.valores_neutro = self.df[self.df['Modelo'] == 'NEUTRO'].iloc[0].dropna()

        # DataFrame para cálculo (exclui linhas de configuração)
        self.calculo_df = self.df.drop([0, 1, 2], errors='ignore').reset_index(drop=True)

        # Converte tipos de dados
        self._convert_data_types()

    def _convert_data_types(self):
        """Converte os tipos de dados conforme especificado"""
        for col in self.tipos_dados.index:
            if col not in self.calculo_df.columns:
                continue

            if self.tipos_dados[col] == 'number':
                self.valores_bom[col] = float(self.valores_bom[col])
                self.valores_neutro[col] = float(self.valores_neutro[col])
                self.calculo_df[col] = pd.to_numeric(self.calculo_df[col], errors='coerce')
            elif self.tipos_dados[col] == 'boolean':
                self.valores_bom[col] = self.valores_bom[col] == 'TRUE'
                self.valores_neutro[col] = self.valores_neutro[col] == 'FALSE'
                self.calculo_df[col] = self.calculo_df[col].map({'TRUE': True, 'FALSE': False, True: True, False: False})
            elif self.tipos_dados[col] == 'string':
                self.calculo_df[col] = self.calculo_df[col].astype(str)

    def calcular_pontuacao_numerica(self, valor, bom, neutro, peso, proporcionalidade_tipo):
        """Calcula pontuação para dados numéricos e booleanos"""
        if bom == neutro:
            return 0

        if isinstance(valor, bool):
            return peso if valor == bom else (-peso if valor != neutro else 0)

        try:
            valor = float(valor)
            bom = float(bom)
            neutro = float(neutro)
            peso = float(peso)
        except (ValueError, TypeError):
            return 0

        if proporcionalidade_tipo == 'proportional':
            if (neutro - bom) == 0:
                return 0
            return peso - ((peso * (valor - bom)) / (neutro - bom))

        elif proporcionalidade_tipo == 'i_proportional':
            if valor <= bom:
                return peso
                # Usa a mesma fórmula mas com lógica invertida
            elif valor > neutro:
                return ((peso * (neutro - valor)) / (neutro - bom))
            else:
                return ((peso * (neutro - valor)) / (neutro - bom))

        elif proporcionalidade_tipo == 'boolean':
            return peso if valor == bom else (-peso if valor != neutro else 0)

        return 0

    def calcular_pontuacao_string(self, valor, coluna):
        """Calcula pontuação para dados de string de forma genérica"""
        peso = self.pesos[coluna]
        valor_bom = self.valores_bom[coluna]
        valor_neutro = self.valores_neutro[coluna]

        # Verifica se há valores específicos para strings (como cores)
        if f"{coluna},pts_string" in self.df.columns:
            valores_string = dict(zip(
                self.df[coluna].dropna(),
                self.df[f"{coluna},pts_string"].dropna().astype(float)
            ))
            if valor in valores_string:
                if valor == valor_bom:
                    return peso * valores_string[valor]
                elif valor == valor_neutro:
                    return 0
                else:
                    return -peso * 0.01  # Penalização padrão para valores não preferidos

        # Lógica genérica para strings sem valores específicos
        if valor == valor_bom:
            return peso
        elif valor == valor_neutro:
            return 0
        return -peso * 0.1

    def calcular_pontuacoes(self):
        """Calcula todas as pontuações"""
        for col in self.pesos.index:
            if col not in self.calculo_df.columns:
                continue

            if self.tipos_dados[col] in ['number', 'boolean']:
                self.calculo_df[f"{col}_pontuacao"] = self.calculo_df.apply(
                    lambda x: self.calcular_pontuacao_numerica(
                        x[col],
                        self.valores_bom[col],
                        self.valores_neutro[col],
                        self.pesos[col],
                        self.proporcionalidade[col]
                    ), axis=1
                )
            elif self.tipos_dados[col] == 'string':
                self.calculo_df[f"{col}_pontuacao"] = self.calculo_df.apply(
                    lambda x: self.calcular_pontuacao_string(x[col], col),
                    axis=1
                )

        # Calcula pontuação total
        colunas_pontuacao = [col for col in self.calculo_df.columns if '_pontuacao' in col]
        self.calculo_df['Pontuacao_Total'] = self.calculo_df[colunas_pontuacao].sum(axis=1)

    def mostrar_pontuacoes(self, apenas_pontuacoes=True):
      """Exibe o DataFrame com as pontuações calculadas

      Args:
          apenas_pontuacoes (bool): Se True, mostra apenas colunas de pontuação
                                    Se False, mostra todas as colunas
      """
      if apenas_pontuacoes:
          # Filtra apenas colunas que terminam com '_pontuacao' ou são 'Modelo' ou 'Pontuacao_Total'
          cols = [col for col in self.calculo_df.columns
                if col.endswith('_pontuacao') or col in ['Modelo', 'Pontuacao_Total']]
          display(self.calculo_df[cols])
      else:
          display(self.calculo_df)

    def gerar_graficos(self):
        """Gera os gráficos de análise"""
        self._gerar_grafico_barras()
        self._gerar_grafico_strings()
        self._gerar_grafico_booleanos()

    def _gerar_grafico_barras(self):
        """Gráfico de barras para pontuação total"""
        df_analise = self.calculo_df[['Modelo', 'Pontuacao_Total']].copy()

        df_analise['Color_Category'] = df_analise.apply(
            lambda x: 'Classificador' if x['Modelo'] in ['BOM', 'NEUTRO'] else
            ('Positiva' if x['Pontuacao_Total'] >= 0 else 'Negativa'),
            axis=1
        )

        color_map = {
            'Classificador': 'cornflowerblue',
            'Positiva': 'lightgreen',
            'Negativa': 'palevioletred'
        }

        df_analise_sorted = df_analise.sort_values('Pontuacao_Total', ascending=False)
        df_analise_sorted.drop(df_analise_sorted[df_analise_sorted['Modelo'].isin(['BOM', 'NEUTRO'])].index, inplace=True)

        fig = px.bar(df_analise_sorted,
                    x='Pontuacao_Total',
                    y='Modelo',
                    orientation='h',
                    color='Color_Category',
                    color_discrete_map=color_map,
                    hover_name='Modelo',
                    title='Pontuação Total dos Modelos')

        fig.update_layout(
            yaxis={'categoryorder': 'array', 'categoryarray': df_analise_sorted['Modelo'].tolist()[::-1]},
            plot_bgcolor='white',
            paper_bgcolor='white',
            legend_title_text='Legenda',
            legend_traceorder='normal',
            xaxis_title='Pontuação Total',
            yaxis_title='Modelo'
        )

        # Linha de referência no zero
        fig.add_vline(x=0, line_dash='dash', line_color='black')

        # Linha de referência no desejável
        fig.add_vline(x=df_analise_sorted['Pontuacao_Total'].max(), line_dash='dash', line_color='black')

        # Linha do Desejável (BOM)
        fig.add_annotation(
            x=df_analise_sorted['Pontuacao_Total'].max(),
            y=len(df_analise_sorted) - 1,
            xref="x",
            yref="y",
            text="Desejável<br>(BOM)",
            showarrow=True,
            font=dict(size=16, color="#000000"),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=0,
            ay=-45,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor='white',
            opacity=1
        )

        # Linha Neutra (Mínimo Aceitável)
        fig.add_annotation(
            x=0,
            y=len(df_analise_sorted) - 1,
            xref="x",
            yref="y",
            text="Mínimo Aceitável<br>(NEUTRO)",
            showarrow=True,
            font=dict(size=16, color="#000000"),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=0,
            ay=-45,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor='white',
            opacity=1
        )

        fig.show()

    def _gerar_grafico_strings(self):
        """Gráfico para variáveis de string"""
        # Identifica colunas de string
        string_cols = [col for col in self.tipos_dados.index
                      if self.tipos_dados[col] == 'string' and col in self.calculo_df.columns]

        if not string_cols:
            return

        # Pega a primeira coluna de string para o gráfico
        col_string = string_cols[0]

        # Cria DataFrame para o gráfico
        df_string = self.calculo_df[['Modelo', col_string]].copy()
        df_string = df_string[~df_string['Modelo'].isin(['BOM', 'NEUTRO'])]

        # Adiciona valores para plotagem
        if f"{col_string},pts_string" in self.df.columns:
            valores_string = dict(zip(
                self.df[col_string].dropna(),
                self.df[f"{col_string},pts_string"].dropna().astype(float)
            ))
            df_string['Valor'] = df_string[col_string].map(valores_string)
        else:
            # Valores padrão se não houver especificação
            unique_vals = df_string[col_string].unique()
            df_string['Valor'] = df_string[col_string].apply(lambda x: list(unique_vals).index(x) * 10)

        # Posição X aleatória para dispersão
        np.random.seed(42)
        df_string['Valor X'] = np.random.normal(0, 1, len(df_string))

        fig = px.scatter(df_string,
                x='Valor X',
                y='Valor',
                color=col_string,
                hover_name='Modelo',
                title=f'Análise de {col_string.capitalize()}',
                size_max=30,  # Aumenta o tamanho máximo das bolinhas
                hover_data={'Valor X': False, 'Valor': False})  # Remove VALOR do hover

        # Linhas de referência
        if f"{col_string},pts_string" in self.df.columns:
            valor_bom = valores_string.get(self.valores_bom[col_string], 0)
            valor_neutro = valores_string.get(self.valores_neutro[col_string], 0)
        else:
            valor_bom = 50  # Valor padrão
            valor_neutro = 25  # Valor padrão

        fig.add_hline(y=valor_bom, line_dash='dash', annotation_text="BOM")
        fig.add_hline(y=valor_neutro, line_dash='dash', annotation_text="NEUTRO")

        fig.update_layout(
            xaxis_title=None,
            xaxis=dict(
                showticklabels=False,
                showgrid=False
            ),
            yaxis_title=None,
            yaxis=dict(
                showticklabels=False,
                showgrid=False
            ),
            plot_bgcolor='white',
            paper_bgcolor='white'
        )

        fig.update_traces(marker=dict(size=10))

        fig.show()

    def _gerar_grafico_booleanos(self):
        """Gráfico para variáveis booleanas"""
        # Identifica colunas booleanas
        bool_cols = [col for col in self.tipos_dados.index
                    if self.tipos_dados[col] == 'boolean' and col in self.calculo_df.columns]

        if not bool_cols:
            return

        # Pega a primeira coluna booleana para o gráfico
        col_bool = bool_cols[0]

        df_bool = self.calculo_df[~self.calculo_df['Modelo'].isin(['BOM', 'NEUTRO'])][['Modelo', col_bool]].copy()
        df_bool['Valor'] = df_bool[col_bool].map({True: 1, False: 0})
        df_bool['Tamanho'] = 20

        fig = px.scatter(df_bool,
                x='Modelo',
                y='Valor',
                color=col_bool,
                color_discrete_map={True: '#4CAF50', False: '#F44336'},
                size='Tamanho',
                title=f'{col_bool} por Modelo',
                hover_name='Modelo',
                hover_data={'Valor': False, 'Tamanho': False},
                category_orders={col_bool: [True, False]})

        # Atualiza a legenda para mostrar SIM/NÃO
        fig.for_each_trace(lambda t: t.update(name='SIM' if t.name == 'True' else 'NÃO'))

        fig.update_yaxes(
            range=[-0.5, 1.5],
            tickvals=[0, 1],
            ticktext=['Não', 'Sim'],
            showgrid=False
        )

        fig.update_layout(
            xaxis_title=None,
            yaxis_title=None,
            plot_bgcolor='white',
            paper_bgcolor='white'
        )

        # Linhas de referência
        valor_bom = 1 if self.valores_bom[col_bool] else 0
        valor_neutro = 1 if self.valores_neutro[col_bool] else 0

        # Verifica se BOM e NEUTRO são iguais
        if valor_bom == valor_neutro:
            fig.add_hline(y=valor_bom,
                        line_dash='dash',
                        line_color='gray',
                        annotation_text="BOM/NEUTRO",
                        annotation_position="top right")
        else:
            fig.add_hline(y=valor_bom,
                        line_dash='dash',
                        line_color='gray',
                        annotation_text="BOM",
                        annotation_position="top right")
            fig.add_hline(y=valor_neutro,
                          line_dash='dash',
                          line_color='gray',
                          annotation_text="NEUTRO",
                          annotation_position="bottom right")

        fig.show()

    def recomendar_produtos(self):
        """Recomenda produtos com base na pontuação"""
        produtos = self.calculo_df[~self.calculo_df['Modelo'].isin(['BOM', 'NEUTRO'])]

        if produtos.empty:
            print("Nenhum produto encontrado para recomendação.")
            return

        pontuacao_maxima = produtos['Pontuacao_Total'].max()
        recomendados = produtos[produtos['Pontuacao_Total'] == pontuacao_maxima]

        print("\nProdutos recomendados:")
        for _, produto in recomendados.iterrows():
            print(f"\nProduto: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
            print("Pontos de vantagem e justificativas:")

            for col in self.pesos.index:
                if col not in self.calculo_df.columns:
                    continue

                pont_col = f"{col}_pontuacao"
                if pont_col not in produto:
                    continue

                pontuacao = produto[pont_col]

                if pontuacao > 0:
                    if self.tipos_dados[col] in ['number', 'boolean']:
                        prop = self.proporcionalidade[col]
                        peso = self.pesos[col]

                        if prop == 'proportional':
                            print(f"- {col}: Performance {pontuacao/peso*100:.1f}% melhor que o mínimo aceitável")
                        elif prop == 'i_proportional':
                            print(f"- {col}: Performance {pontuacao/peso*100:.1f}% melhor que o mínimo aceitável (inversamente proporcional)")
                        else:
                            print(f"- {col}: Atende ao critério desejado")

                    elif self.tipos_dados[col] == 'string':
                        print(f"- {col}: Valor '{produto[col]}' é preferido")

                elif pontuacao < 0:
                    if self.tipos_dados[col] in ['number', 'boolean']:
                        prop = self.proporcionalidade[col]
                        peso = self.pesos[col]

                        if prop == 'proportional':
                            print(f"- {col}: Performance {-pontuacao/peso*100:.1f}% pior que o mínimo aceitável")
                        elif prop == 'i_proportional':
                            print(f"- {col}: Performance {-pontuacao/peso*100:.1f}% pior que o mínimo aceitável (inversamente proporcional)")
                        else:
                            print(f"- {col}: Não atende ao critério mínimo")

                    elif self.tipos_dados[col] == 'string':
                        print(f"- {col}: Valor '{produto[col]}' não é o preferido")

# Exemplo de uso para a base de carros
print("Análise para base de carros:")
analyzer_carros = DataAnalyzer('base_carro.csv')
analyzer_carros.load_and_prepare_data()
analyzer_carros.calcular_pontuacoes()
analyzer_carros.gerar_graficos()
analyzer_carros.recomendar_produtos()

Análise para base de carros:



Produtos recomendados:

Produto: Hyundai HB20 com pontuação total de 0.52
Pontos de vantagem e justificativas:
- Preço (R$): Performance 52.0% melhor que o mínimo aceitável (inversamente proporcional)
- Consumo (km/l): Performance 76.0% melhor que o mínimo aceitável
- Potência (cv): Performance 36.7% melhor que o mínimo aceitável
- Segurança (estrelas): Performance 50.0% melhor que o mínimo aceitável
- Ar Condicionado: Atende ao critério desejado
- Cor: Valor 'Branco' é preferido


In [81]:
analyzer_carros.mostrar_pontuacoes(apenas_pontuacoes=True)

,Modelo,Preço (R$)_pontuacao,Consumo (km/l)_pontuacao,Potência (cv)_pontuacao,Porta-malas (litros)_pontuacao,Segurança (estrelas)_pontuacao,Ar Condicionado_pontuacao,Câmbio Automático_pontuacao,Cor_pontuacao,Pontuacao_Total
0,BOM,0.300,0.100,0.100000,0.0500,0.2,0.1,0,0.050,0.900000
1,NEUTRO,0.000,0.000,0.000000,0.0000,0.0,0.1,0,0.000,0.100000
2,Fiat Argo,0.120,0.070,0.063333,0.0250,0.1,0.1,0,-0.005,0.473333
3,Volkswagen Polo,0.036,0.056,0.086667,0.0100,0.2,0.1,0,-0.005,0.483667
4,Chevrolet Onix,0.084,0.084,0.053333,0.0025,0.1,0.1,0,-0.005,0.418833
5,Hyundai HB20,0.156,0.076,0.036667,0.0000,0.1,0.1,0,0.050,0.518667
6,Renault Kwid,0.240,0.110,-0.066667,-0.0050,0.0,0.1,0,0.000,0.378333
7,Toyota Yaris,-0.036,0.050,0.066667,0.0050,0.1,0.1,0,-0.005,0.280667
8,Honda City,-0.084,0.036,0.120000,0.1000,0.2,0.1,0,-0.005,0.467000
9,Peugeot 208,0.060,0.060,0.076667,0.0075,0.1,0.1,0,-0.005,0.399167
